list of updates:

1.   incorporated ravel(.) into predictinfill(.) to deal with error message
2.   updated predictinfill(.) scikit architectures to RandomForest instead of support vector (10/8 7:44pm)
3. change all nmbr_dict to bxcx_dict for boxcox process (10/8 8:06pm)
4. add features to column_dict for bnry and nmbr (10/8 8:15pm)
5. initialize nmbr and bnry dict's, add entries to postprocess_dict (10/8 8:30pm)
6. update process functions to output columnslist and update column_dict where this list was #'d out', then update automunge to allow for multicolumn transformations in preprocessing stage (10/8 9:25pm)
7. incorporate infillcomplete process for multicolumn transformations to automunge ML infill (10/8 9:48pm)
8. incorporate NArows into processing functions for bxcx, nmbr, bnry, date (one at a time -> test ML infill each time, I think there might be some complexity in the create infill sets function - update complexity resolved by using different label for NArw column) (10/9 2:20pm)
9. convert binary integers to 8 bit (10/9 2:42pm)
10.  initialize multicolumntransform_dict prior to processing (10/11, 10:19 am)
11. have process functions output an additional list categorylist (for now blank lists except for text class) (10/11 10:32am)
12. update process steps for nmbr and bnry to include for column in columnslist address of column_dict and nmbr_dict, note this leads to bug in createinfillsets need to troubleshoot (10-11 7:26pm)
13. update MLinfill functions creatMLinfillsets() and insertinfill()  to generalize between categories to categorical and single column (10-11 3:18pm) (10-11 7:26pm)
14. audit column_dict and category dicts and postprocess_dict -> update to generalized format between columns (major architecture update) (10-11 10:30pm)
15. combine category dict's into single MLinfill address for this generalization (10-12 10:30am)
16. update postprocess functions and postinfill functions (10/12 2:24pm) (10/12 8:30pm)
17. update postmunge (10/12 4:11pm) (10/12 8:30pm)

complete pending troubleshoot:

remaining to address:

- labels processing at end of automunge
- numerically encoded categorical data
- carve out ML infill application from automunge to seperate functions
- generalize MLinfill to columns with multiple transforms of different types (bxcx  is good testing ground)
- double check createMLinfill sets has no leakage
- double check  correct category applied for the application of ML infill




In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#for measuring time duration of operations
from timeit import default_timer as timer

#for file downloads
import pickle
from google.colab import files

# 1) insert preprocess functions from prior notebook. 

In [0]:
#process_numerical_class(mdf_train, mdf_test, column)
#function to normalize data to mean of 0 and standard deviation of 1 from training distribution
#takes as arguement pandas dataframe of training and test data (mdf_train), (mdf_test)\
#and the name of the column string ('column') 
#replaces missing or improperly formatted data with mean of remaining values
#replaces original specified column in dataframe
#returns transformed dataframe

#expect this approach works better when the numerical distribution is thin tailed
#if only have training but not test data handy, use same training data for both dataframe inputs

#imports
from pandas import Series
from sklearn import preprocessing

def process_numerical_class(mdf_train, mdf_test, column):
     
  
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_nmbr_train = NArows(mdf_train, column, 'nmbr')
  NArows_nmbr_test = NArows(mdf_test, column, 'nmbr')
  mdf_train[column + '_NArw'] = NArows_nmbr_train.copy()
  mdf_test[column + '_NArw'] = NArows_nmbr_test.copy()
  del NArows_nmbr_train
  del NArows_nmbr_test
  
  #change NArows data type to 8-bit (1 byte) integers for memory savings
  mdf_train[column + '_NArw'] = mdf_train[column + '_NArw'].astype(np.int8)
  mdf_test[column + '_NArw'] = mdf_test[column + '_NArw'].astype(np.int8)
  
  #convert all values to either numeric or NaN
  mdf_train[column] = pd.to_numeric(mdf_train[column], errors='coerce')
  mdf_test[column] = pd.to_numeric(mdf_test[column], errors='coerce')

  #get mean of training data
  mean = mdf_train[column].mean()    

  #replace missing data with training set mean
  mdf_train[column] = mdf_train[column].fillna(mean)
  mdf_test[column] = mdf_test[column].fillna(mean)

  #subtract mean from column for both train and test
  mdf_train[column] = mdf_train[column] - mean
  mdf_test[column] = mdf_test[column] - mean

  #get standard deviation of training data
  std = mdf_train[column].std()

  #divide column values by std for both training and test data
  mdf_train[column] = mdf_train[column] / std
  mdf_test[column] = mdf_test[column] / std
  
  #change column name to column + '_nmbr'
  mdf_train[column + '_nmbr'] = mdf_train[column].copy()
  mdf_test[column + '_nmbr'] = mdf_test[column].copy()
  del mdf_train[column]
  del mdf_test[column]
  
  #create list of columns
  nmbrcolumns = [column + '_nmbr', column + '_NArw']
  
  #create list of columns associated with categorical transform (blank for now)
  categorylist = []


  return mdf_train, mdf_test, mean, std, nmbrcolumns, categorylist
  

  
#process_binary_class(mdf, column, missing)
#converts binary classification values to 0 or 1
#takes as arguement a pandas dataframe (mdf), \
#the name of the column string ('column') \
#and the string classification to assign to missing data ('missing')
#replaces original specified column in dataframe
#returns transformed dataframe

#missing category must be identical to one of the two existing categories
#returns error message if more than two categories remain


def process_binary_class(mdf, column, missing):
    
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_bnry = NArows(mdf, column, 'bnry')
  mdf[column + '_NArw'] = NArows_bnry.copy()
  del NArows_bnry
  
  #replace missing data with specified classification
  mdf[column] = mdf[column].fillna(missing)

  #if more than two remaining classifications, return error message    
  if len(mdf[column].unique()) > 2:
      print('ERROR: number of categories in column for process_binary_class() call >2')
      return mdf

  #convert column to binary 0/1 classification
  lb = preprocessing.LabelBinarizer()
  mdf[column] = lb.fit_transform(mdf[column])

  #change column name to column + '_bnry'
  mdf[column + '_bnry'] = mdf[column].copy()
  del mdf[column]
  
  #create list of columns
  bnrycolumns = [column + '_bnry', column + '_NArw']
  
  #change data types to 8-bit (1 byte) integers for memory savings
  for bnrycolumn in bnrycolumns:
    mdf[bnrycolumn] = mdf[bnrycolumn].astype(np.int8)
    
  #create list of columns associated with categorical transform (blank for now)
  categorylist = []
  
  return mdf, bnrycolumns, categorylist

  
#process_text_class(mdf_train, mdf_test, column)
#preprocess column with text classifications
#takes as arguement two pandas dataframe containing training and test data respectively 
#(mdf_train, mdf_test), and the name of the column string ('column')

#note this trains both training and test data simultaneously due to unique treatment if any category
#missing from training set but not from test set to ensure consistent formatting 

#deletes the original column from master dataframe and
#replaces with onehot encodings
#with columns named after column_ + text classifications
#missing data replaced with category label 'missing'+column
#any categories missing from the training set removed from test set
#any category present in training but missing from test set given a column of zeros for consistent formatting
#ensures order of all new columns consistent between both sets
#returns two transformed dataframe (mdf_train, mdf_test) \
#and a list of the new column names (textcolumns)

#if only have training but not test data handy, use same training data for both dataframe inputs

#note it is kind of a hack here to create a column for missing values with \
#two underscores (__) in the column name to ensure appropriate order for cases\
#where NaN present in test data but not train data, if a category starts with|
#an underscore such that it preceeds '__missing' alphabetically in this scenario\
#this might create error due to different order of columns, address of this \
#potential issue will be a future extension


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def process_text_class(mdf_train, mdf_test, column):

  
  
  #replace NA with a dummy variable
  mdf_train[column] = mdf_train[column].fillna('NArw')
  mdf_test[column] = mdf_test[column].fillna('NArw')


  #extract categories for column labels
  #note that .unique() extracts the labels as a numpy array
  labels_train = mdf_train[column].unique()
  labels_train.sort(axis=0)
  labels_test = mdf_test[column].unique()
  labels_test.sort(axis=0)

  #transform text classifications to numerical id
  encoder = LabelEncoder()
  cat_train = mdf_train[column]
  cat_train_encoded = encoder.fit_transform(cat_train)

  cat_test = mdf_test[column]
  cat_test_encoded = encoder.fit_transform(cat_test)


  #apply onehotencoding
  onehotencoder = OneHotEncoder()
  cat_train_1hot = onehotencoder.fit_transform(cat_train_encoded.reshape(-1,1))
  cat_test_1hot = onehotencoder.fit_transform(cat_test_encoded.reshape(-1,1))

  #append column header name to each category listing
  #note the iteration is over a numpy array hence the [...] approach  
  labels_train[...] = column + '_' + labels_train[...]
  labels_test[...] = column + '_' + labels_test[...]


  #convert sparse array to pandas dataframe with column labels
  df_train_cat = pd.DataFrame(cat_train_1hot.toarray(), columns=labels_train)
  df_test_cat = pd.DataFrame(cat_test_1hot.toarray(), columns=labels_test)
  
  #add a missing column to train if it's not present
  if column + '_NArw' not in df_train_cat.columns:
    missingcolumn = pd.DataFrame(0, index=np.arange(df_train_cat.shape[0]), columns=[column+'_NArw'])
    df_train_cat = pd.concat([missingcolumn, df_train_cat], axis=1)


  #Get missing columns in test set that are present in training set
  missing_cols = set( df_train_cat.columns ) - set( df_test_cat.columns )
  
  #Add a missing column in test set with default value equal to 0
  for c in missing_cols:
      df_test_cat[c] = 0
  #Ensure the order of column in the test set is in the same order than in train set
  #Note this also removes categories in test set that aren't present in training set
  df_test_cat = df_test_cat[df_train_cat.columns]


  #concatinate the sparse set with the rest of our training data
  mdf_train = pd.concat([df_train_cat, mdf_train], axis=1)
  mdf_test = pd.concat([df_test_cat, mdf_test], axis=1)


  #delete original column from training data
  del mdf_train[column]    
  del mdf_test[column]
  
  #create output of a list of the created column names
  labels_train = list(df_train_cat)
  textcolumns = labels_train
  
  #change data types to 8-bit (1 byte) integers for memory savings
  for textcolumn in textcolumns:
    mdf_train[textcolumn] = mdf_train[textcolumn].astype(np.int8)
    mdf_test[textcolumn] = mdf_test[textcolumn].astype(np.int8)
  
  #create list of columns associated with categorical transform
  categorylist = textcolumns

  return mdf_train, mdf_test, textcolumns, categorylist



#process_time_class(mdf_train, mdf_test, column)
#preprocess column with time classifications
#takes as arguement two pandas dataframe containing training and test data respectively 
#(mdf_train, mdf_test), and the name of the column string ('column')

#note this trains both training and test data simultaneously due to unique treatment if any category
#missing from training set but not from test set to ensure consistent formatting 

#deletes the original column from master dataframe and
#replaces with distinct columns for year, month, day, hour, minute, second
#each normalized to the mean and std, with missing values plugged with the mean
#with columns named after column_ + time category
#returns two transformed dataframe (mdf_train, mdf_test)

#if only have training but not test data handy, use same training data for both dataframe inputs

import datetime as dt

def process_time_class(mdf_train, mdf_test, column):
  
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_nmbr_train = NArows(mdf_train, column, 'nmbr')
  NArows_nmbr_test = NArows(mdf_test, column, 'nmbr')
  mdf_train[column + '_NArw'] = NArows_nmbr_train.copy()
  mdf_test[column + '_NArw'] = NArows_nmbr_test.copy()
  del NArows_nmbr_train
  del NArows_nmbr_test
  
  #change NArows data type to 8-bit (1 byte) integers for memory savings
  mdf_train[column + '_NArw'] = mdf_train[column + '_NArw'].astype(np.int8)
  mdf_test[column + '_NArw'] = mdf_test[column + '_NArw'].astype(np.int8)
  
  #apply pd.to_datetime to column, note that the errors = 'coerce' needed for messy data
  mdf_train[column] = pd.to_datetime(mdf_train[column], errors = 'coerce')
  mdf_test[column] = pd.to_datetime(mdf_test[column], errors = 'coerce')
  
  #mdf_train[column].replace(-np.Inf, np.nan)
  #mdf_test[column].replace(-np.Inf, np.nan)
  
  #get mean of various categories of datetime objects to use to plug in missing cells
  meanyear = mdf_train[column].dt.year.mean()    
  meanmonth = mdf_train[column].dt.month.mean()
  meanday = mdf_train[column].dt.day.mean()
  meanhour = mdf_train[column].dt.hour.mean()
  meanminute = mdf_train[column].dt.minute.mean()
  meansecond = mdf_train[column].dt.second.mean()
  
  #get standard deviation of training data
  stdyear = mdf_train[column].dt.year.std()  
  stdmonth = mdf_train[column].dt.month.std()
  stdday = mdf_train[column].dt.day.std()
  stdhour = mdf_train[column].dt.hour.std()
  stdminute = mdf_train[column].dt.minute.std()
  stdsecond = mdf_train[column].dt.second.std()
  
  
  #create new columns for each category in train set
  mdf_train[column + '_year'] = mdf_train[column].dt.year
  mdf_train[column + '_month'] = mdf_train[column].dt.month
  mdf_train[column + '_day'] = mdf_train[column].dt.day
  mdf_train[column + '_hour'] = mdf_train[column].dt.hour
  mdf_train[column + '_minute'] = mdf_train[column].dt.minute
  mdf_train[column + '_second'] = mdf_train[column].dt.second
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column].dt.year
  mdf_test[column + '_month'] = mdf_test[column].dt.month
  mdf_test[column + '_day'] = mdf_test[column].dt.day
  mdf_test[column + '_hour'] = mdf_test[column].dt.hour
  mdf_test[column + '_minute'] = mdf_test[column].dt.minute 
  mdf_test[column + '_second'] = mdf_test[column].dt.second
  

  #replace missing data with training set mean
  mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(meanyear)
  mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(meanmonth)
  mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(meanday)
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(meanhour)
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(meanminute)
  mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(meansecond)
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(meanyear)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(meanmonth)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(meanday)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(meanhour)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(meanminute)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(meansecond)
  
  #subtract mean from column for both train and test
  mdf_train[column + '_year'] = mdf_train[column + '_year'] - meanyear
  mdf_train[column + '_month'] = mdf_train[column + '_month'] - meanmonth
  mdf_train[column + '_day'] = mdf_train[column + '_day'] - meanday
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'] - meanhour
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'] - meanminute
  mdf_train[column + '_second'] = mdf_train[column + '_second'] - meansecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] - meanyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] - meanmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] - meanday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] - meanhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] - meanminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] - meansecond
  
  
  #divide column values by std for both training and test data
  mdf_train[column + '_year'] = mdf_train[column + '_year'] / stdyear
  mdf_train[column + '_month'] = mdf_train[column + '_month'] / stdmonth
  mdf_train[column + '_day'] = mdf_train[column + '_day'] / stdday
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'] / stdhour
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'] / stdminute
  mdf_train[column + '_second'] = mdf_train[column + '_second'] / stdsecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] / stdyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] / stdmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] / stdday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] / stdhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] / stdminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] / stdsecond
  
  
  #now replace NaN with 0
  mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(0)
  mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(0)
  mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(0)
  mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(0)
  mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(0)
  mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(0)
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(0)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(0)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(0)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(0)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(0)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(0)
  
  #output of a list of the created column names
  datecolumns = [column + '_year', column + '_month', column + '_day', \
                column + '_hour', column + '_minute', column + '_second', \
                column + '_NArw']
  
  #this is to address an issue I found when parsing columns with only time no date
  #which returned -inf vlaues, so if an issue will just delete the associated 
  #column along with the entry in datecolumns
  checkyear = np.isinf(mdf_train.iloc[0][column + '_year'])
  if checkyear:
    del mdf_train[column + '_year']
    datecolumns.remove(column + '_year')
    if column + '_year' in mdf_test.columns:
      del mdf_test[column + '_year']

  checkmonth = np.isinf(mdf_train.iloc[0][column + '_month'])
  if checkmonth:
    del mdf_train[column + '_month']
    datecolumns.remove(column + '_month')
    if column + '_month' in mdf_test.columns:
      del mdf_test[column + '_month']

  checkday = np.isinf(mdf_train.iloc[0][column + '_day'])
  if checkday:
    del mdf_train[column + '_day']
    datecolumns.remove(column + '_day')
    if column + '_day' in mdf_test.columns:
      del mdf_test[column + '_day']
  
  
  #delete original column from training data
  del mdf_train[column]    
  if column in mdf_test.columns:
    del mdf_test[column]  
  

  
  #output a dictionary of the associated column mean and std
  
  timenormalization_dict = {'meanyear' : meanyear, 'meanmonth' : meanmonth, \
                            'meanday' : meanday, 'meanhour' : meanhour, \
                            'meanminute' : meanminute, 'meansecond' : meansecond,\
                            'stdyear' : stdyear, 'stdmonth' : stdmonth, \
                            'stdday' : stdday, 'stdhour' : stdhour, \
                            'stdminute' : stdminute, 'stdsecond' : stdsecond}
  
  #create list of columns associated with categorical transform (blank for now)
  categorylist = []
  
  return mdf_train, mdf_test, datecolumns, timenormalization_dict, categorylist
  
  

# 2) Create new preprocess function for Box-Cox transformation

In [0]:
#process_bxcx_class(df, column, bxcx_lmbda = None, trnsfrm_mean = None, trnsfrm_std = None)
#function that takes as input a dataframe with numnerical column for purposes
#of applying a box-cox transformation. If lmbda = None it will infer a suitable
#lambda value by minimizing log likelihood using SciPy's stats boxcox call. If
#we pass a mean or std value it will apply the mean for the initial infill and 
#use the values to apply postprocess_numerical_class function. 
#Returns transformed dataframe, a list nmbrcolumns of the associated columns,
#and a normalization dictionary nmbrnormalization_dict which we'll use for our
#postprocess_dict, and the parameter lmbda that was used

#expect this approach works better than our prior numerical address when the 
#distribution is less thin tailed


#imports
from scipy import stats

def process_bxcx_class(df, column, bxcx_lmbda = None, trnsfrm_mean = None, \
                       trnsfrm_std = None):
  
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_bxcx = NArows(df, column, 'nmbr')
  df[column + '_NArw'] = NArows_bxcx.copy()
  del NArows_bxcx
  
    
  #change NArows data type to 8-bit (1 byte) integers for memory savings
  df[column + '_NArw'] = df[column + '_NArw'].astype(np.int8)
  
  #convert all values to either numeric or NaN
  df[column] = pd.to_numeric(df[column], errors='coerce')
  
  #get the mean value to apply to infill
  if trnsfrm_mean == None:
    #get mean of training data
    mean = df[column].mean()  
    
  else:
    mean = trnsfrm_mean

  #replace missing data with training set mean
  df[column] = df[column].fillna(mean)
  
  #apply box-cox transformation to generate a new column
  #note the returns are different based on whether we passed a lmbda value
  
  if bxcx_lmbda == None:
    
    df[column + '_bxcx'], bxcx_lmbda = stats.boxcox(df[column])
    
  else:
    
    df[column + '_bxcx'] = stats.boxcox(df[column], lmbda = bxcx_lmbda)
  
  
  #apply process_numerical_class to other column and change name
  #although if we were given a mean and std value then apply postprocess function
  
  #if std is none
  if trnsfrm_std == None:
    
    #get std from train set
    std = df[column].std()
    
    #apply numerical transforms
    df[column + '_nmbr'] = df[column] - mean
    df[column + '_nmbr'] = df[column + '_nmbr'] / std
    
  else:
    
    std = trnsfrm_std
    
    #apply numerical transforms
    df[column + '_nmbr'] = df[column] - mean
    df[column + '_nmbr'] = df[column + '_nmbr'] / std 

  #delete the original column
  del df[column]  
  
  
  #output of a list of the created column names
  nmbrcolumns = [column + '_nmbr', column + '_bxcx', column + '_NArw']
  
  #output a dictionary of the associated column mean and std
  
  nmbrnormalization_dict = {'trnsfrm_mean' : mean, 'trnsfrm_std' : std, \
                            'bxcx_lmbda' : bxcx_lmbda}
  
  #create list of columns associated with categorical transform (blank for now)
  categorylist = []
  
  return df, nmbrcolumns, nmbrnormalization_dict, categorylist

# 3) Update evalcategory function for new bxcx category

In [0]:
#evalcategory(df, column)
#Function that dakes as input a dataframe and associated column id \
#evaluates the contents of cells and classifies the column into one of four categories
#category 1, 'bnry', is for columns with only two categorys of text or integer
#category 2, 'nmbr', is for columns with numerical integer or float values
#category 3: 'bxcx', is for nmbr category with all positive values
#category 4, 'text', is for columns with multiple categories appropriate for one-hot
#category 5, 'date', is for columns with Timestamp data
#category 6, 'null', is for columns with >85% null values (arbitrary figure)
#returns category id as a string

import collections
import datetime as dt

def evalcategory(df, column):

  
  #I couldn't find a good pandas tool for evaluating data class, \
  #So will produce an array containing data types of each cell and \
  #evaluate for most common variable using the collections library
  
  type1_df = df[column].apply(lambda x: type(x)).values
  
  c = collections.Counter(type1_df)
  mc = c.most_common(1)
  mc2 = c.most_common(2)
  
  #free memory (dtypes are memory hogs)
  type1_df = None

  
  #additional array needed to check for time series
  
  #df['typecolumn2'] = df[column].apply(lambda x: type(pd.to_datetime(x, errors = 'coerce')))
  type2_df = df[column].apply(lambda x: type(pd.to_datetime(x, errors = 'coerce'))).values
  
  datec = collections.Counter(type2_df)
  datemc = datec.most_common(1)
  datemc2 = datec.most_common(2)
  
  #free memory (dtypes are memory hogs)
  type2_df = None
  
  
  #an extension of this approach could be for those columns that produce a text\
  #category to implement an additional text to determine the number of \
  #common groupings / or the amount of uniquity. For example if every row has\
  #a unique value then one-hot-encoding would not be appropriate. It would \
  #probably be apopropraite to either return an error message if this is found \
  #or alternatively find a furhter way to automate this processing such as \
  #look for contextual clues to groupings that can be inferred.
    
  #This is kind of hack to evaluate class by comparing these with output of mc
  checkint = 1
  checkfloat = 1.1
  checkstring = 'string'
  checkNAN = None

  #there's probably easier way to do this, here will create a check for date
  df_checkdate = pd.DataFrame([{'checkdate' : '7/4/2018'}])
  df_checkdate['checkdate'] = pd.to_datetime(df_checkdate['checkdate'], errors = 'coerce')
  

  #create dummy variable to store determined class (default is text class)
  category = 'text'


  #if most common in column is string and > two values, set category to text
  if isinstance(checkstring, mc[0][0]) and df[column].nunique() > 2:
    category = 'text'
  
  #if most common is date, set category to date
  if isinstance(df_checkdate['checkdate'][0], datemc[0][0]):
    category = 'date'
  
  #if most common in column is integer and > two values, set category to number of bxcx
  if isinstance(checkint, mc[0][0]) and df[column].nunique() > 2:
    
    #if all postiive set category to bxcx
    
    #if (pd.to_numeric(df[column], errors = 'coerce').notnull() >= 0).all():
    
    #note we'll only allow bxcx category if all values greater than a clip value
    #>0 (currently set at 0.001)since there is an asymptote for box-cox at 0
    if (df[pd.to_numeric(df[column], errors='coerce').notnull()][column] >= 0.001).all():
      category = 'bxcx'
    
    else:
      category = 'nmbr'
    
  #if most common in column is float, set category to number or bxcx
  if isinstance(checkfloat, mc[0][0]):
    
    #if all postiive set category to bxcx
    
    #note we'll only allow bxcx category if all values greater than a clip value
    #>0 (currently set at 0.001) since there is an asymptote for box-cox at 0
    if (df[pd.to_numeric(df[column], errors='coerce').notnull()][column] >= 0.001).all():
      category = 'bxcx'
    
    else:
      category = 'nmbr'
  
  #if most common in column is integer and <= two values, set category to binary
  if isinstance(checkint, mc[0][0]) and df[column].nunique() <= 2:
    category = 'bnry'
  
  #if most common in column is string and <= two values, set category to binary
  if isinstance(checkstring, mc[0][0]) and df[column].nunique() <= 2:
    category = 'bnry'
    
      
  #if > 80% (ARBITRARY FIGURE) are NaN we'll just delete the column
  if df[column].isna().sum() >= df.shape[0] * 0.80:
    category = 'null'
  
  #else if most common in column is NaN, re-evaluate using the second most common type
  #(I suspect the below might have a bug somewhere but is working on my current 
  #tests so will leave be for now)
  elif df[column].isna().sum() >= df.shape[0] / 2:
    
    #if 2nd most common in column is string and > two values, set category to text
    if isinstance(checkstring, mc2[1][0]) and df[column].nunique() > 2:
      category = 'text'
  
    #if 2nd most common is date, set category to date   
    if isinstance(df_checkdate['checkdate'][0], datemc2[0][0]):
      category = 'date'
  
    #if 2nd most common in column is integer and > two values, set category to number
    if isinstance(checkint, mc2[1][0]) and df[column].nunique() > 2:
      category = 'nmbr'
    
    #if 2nd most common in column is float, set category to number
    if isinstance(checkfloat, mc2[1][0]):
      category = 'nmbr'
  
    #if 2nd most common in column is integer and <= two values, set category to binary
    if isinstance(checkint, mc2[1][0]) and df[column].nunique() <= 2:
      category = 'bnry'
  
    #if 2nd most common in column is string and <= two values, set category to binary
    if isinstance(checkstring, mc2[1][0]) and df[column].nunique() <= 2:
      category = 'bnry'

  
  return category

# 4) update MLinfill support functions to address new category

In [0]:
#NArows(df, column), function that when fed a dataframe, \
#column id, and category label outputs a single column dataframe composed of \
#True and False with the same number of rows as the input and the True's \
#coresponding to those rows of the input that had missing or NaN data. This \
#output can later be used to identify which rows for a column to infill with ML\
# derived plug data


def NArows(df, column, category):
  

  
  if category == 'text':
  
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
  
  if category == 'bnry':
    
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
    
  if category == 'nmbr' or category == 'bxcx':
  
    #convert all values to either numeric or NaN
    df[column] = pd.to_numeric(df[column], errors='coerce')
    
    #returns dataframe of True and False, where True coresponds to the NaN's
    #renames column name to column + '_NArows'
    NArows = pd.isna(df[column])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})

  
  if category == 'date':
    
    #returns dataframe column of all False
    #renames column name to column + '_NArows'
    NArows = pd.DataFrame(False, index=np.arange(df.shape[0]), columns=[column+'NA'])
    NArows = pd.DataFrame(NArows)
    NArows = NArows.rename(columns = {column:column+'_NArows'})
  

  return NArows





#labelbinarizercorrect(npinput, columnslist), function that takes as input the output\
#array from scikit learn's LabelBinarizer() and ensures that the re-encoding is\
#consistent with the original array prior to performing the argmax. This is \
#needed because LabelBinarizer automatically takes two class sets to a binary\
#setting and doesn't account for columns above index of active values based on\
#my understanding. For a large enough dataset this probably won't be an issue \
#but just trying to be thorough. Outputs a one-hot encoded array comparable to \
#the format of our input to argmax.

def labelbinarizercorrect(npinput, columnslist):
  
  
  #if our array post application of LabelBinarizer has few coloumns than our \
  #column list then run through these loops
  if npinput.shape[1] < len(columnslist):
    
    #if only one column in our array means LabelEncoder must have binarized \
    #since we already established that there are more columns
    if npinput.shape[1] == 1:
      
      #this transfers from the binary encoding to two columns of one hot
      npinput = np.hstack((1 - npinput, npinput))
      
      np_corrected = npinput
      
    #if we still have fewer columns than the column list, means we'll need to \
    #pad out with columns containing zeros
    if npinput.shape[1] < len(columnslist):
      missingcols = len(columnslist) - npinput.shape[1]
      append = np.zeros((npinput.shape[0], missingcols))
      np_corrected = np.concatenate((npinput, append), axis=1)
  
  else:
    #otherwise just return the input array because it is in good shape
    np_corrected = npinput

  
  return np_corrected



#predictinfill(category, df_train_filltrain, df_train_filllabel, \
#df_train_fillfeatures, df_test_fillfeatures, randomseed, columnslist), \
#function that takes as input \
#a category string, the output of createMLinfillsets(.), a seed for randomness \
#and a list of columns produced by a text class preprocessor when applicable and 
#returns predicted infills for the train and test feature sets as df_traininfill, \
#df_testinfill based on derivations using scikit-learn, with the lenth of \
#infill consistent with the number of True values from NArows, and the trained \
#model


#imports for numerical class training
#from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import PassiveAggressiveRegressor
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import RidgeCV
#from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

#imports for binary and text class training
from sklearn import preprocessing
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import SGDClassifier
#from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier


def predictinfill(category, df_train_filltrain, df_train_filllabel, \
                  df_train_fillfeatures, df_test_fillfeatures, randomseed, \
                  columnslist = []):

  #a reasonable extension of this funciton would be to allow ML inference with \
  #other ML architectures such a SVM or something SGD based for instance
  
  #convert dataframes to numpy arrays
  np_train_filltrain = df_train_filltrain.values
  np_train_filllabel = df_train_filllabel.values
  np_train_fillfeatures = df_train_fillfeatures.values
  np_test_fillfeatures = df_test_fillfeatures.values
  
  #ony run the following if we have any rows needing infill
  if df_train_fillfeatures.shape[0] > 0:

    if category == 'nmbr':
      
      #this is to address a weird error message suggesting I reshape the y with ravel()
      np_train_filllabel = np.ravel(np_train_filllabel)
      
      #train linear regression model using scikit-learn for numerical prediction
      #model = LinearRegression()
      #model = PassiveAggressiveRegressor(random_state = randomseed)
      #model = Ridge(random_state = randomseed)
      #model = RidgeCV()
      #note that SVR doesn't have an argument for random_state
      #model = SVR()
      model = RandomForestRegressor(random_state = randomseed)
      
      model.fit(np_train_filltrain, np_train_filllabel)    
      
      
      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])

    
    if category == 'bxcx':
      
      #this is to address a weird error message suggesting I reshape the y with ravel()
      np_train_filllabel = np.ravel(np_train_filllabel)
      
      #model = SVR()
      model = RandomForestRegressor(random_state = randomseed)
      
      model.fit(np_train_filltrain, np_train_filllabel)   
      
      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])     
      
    

    if category == 'bnry':
      
      #this is to address a weird error message suggesting I reshape the y with ravel()
      np_train_filllabel = np.ravel(np_train_filllabel)
      
      #train logistic regression model using scikit-learn for binary classifier
      #model = LogisticRegression()
      #model = LogisticRegression(random_state = randomseed)
      #model = SGDClassifier(random_state = randomseed)
      #model = SVC(random_state = randomseed)
      model = RandomForestClassifier(random_state = randomseed)
      
      model.fit(np_train_filltrain, np_train_filllabel)

      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])

#       print('category is bnry, df_traininfill is')
#       print(df_traininfill)

    if category == 'text':

      #first convert the one-hot encoded set via argmax to a 1D array
      np_train_filllabel_argmax = np.argmax(np_train_filllabel, axis=1)

      #train logistic regression model using scikit-learn for binary classifier
      #with multi_class argument activated
      #model = LogisticRegression()
      #model = SGDClassifier(random_state = randomseed)
      #model = SVC(random_state = randomseed)
      model = RandomForestClassifier(random_state = randomseed)
      
      model.fit(np_train_filltrain, np_train_filllabel_argmax)

      #predict infill values
      np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        #this needs to have same number of columns as text category
        np_testinfill = np.zeros(shape=(1,len(columnslist)))

      #convert the 1D arrary back to one hot encoding
      labelbinarizertrain = preprocessing.LabelBinarizer()
      labelbinarizertrain.fit(np_traininfill)
      np_traininfill = labelbinarizertrain.transform(np_traininfill)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        labelbinarizertest = preprocessing.LabelBinarizer()
        labelbinarizertest.fit(np_testinfill)
        np_testinfill = labelbinarizertest.transform(np_testinfill)



      #run function to ensure correct dimensions of re-encoded classifier array
      np_traininfill = labelbinarizercorrect(np_traininfill, columnslist)
      
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = labelbinarizercorrect(np_testinfill, columnslist)


      #convert infill values to dataframe
      df_traininfill = pd.DataFrame(np_traininfill, columns = [columnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [columnslist]) 
      

#       print('category is text, df_traininfill is')
#       print(df_traininfill)

    if category == 'date':

      #create empty sets for now
      #an extension of this method would be to implement a comparable infill \
      #method for the time category, based on the columns output from the \
      #preprocessing
      df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 
      
      model = False

#       print('category is text, df_traininfill is')
#       print(df_traininfill)
  
  
  #else if we didn't have any infill rows let's create some plug values
  else:
    
    if category == 'text':
      np_traininfill = np.zeros(shape=(1,len(columnslist)))
      np_testinfill = np.zeros(shape=(1,len(columnslist)))
      df_traininfill = pd.DataFrame(np_traininfill, columns = [columnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [columnslist]) 
    
    else :
      df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 
    
    #set model to False, this will be be needed for this eventiality in 
    #test set post-processing
    model = False
  
  return df_traininfill, df_testinfill, model



# Update some of the ML infill functions

In [0]:
#update createMLinfillsets as follows:
#instead of diferientiation by category, do a test for whether categorylist = []
#if so do a single column transform excluding those other columns from columnslist
#in the sets comparable to , otherwise do a transform comparable to text category

#createMLinfillsets(df_train, df_test, column, trainNArows, testNArows, \
#category, columnslist = []) function that when fed dataframes of train and\
#test sets, column id, df of True/False corresponding to rows from original \
#sets with missing values, a string category of 'text', 'date', 'nmbr', or \
#'bnry', and a list of column id's for the text category if applicable. The \
#function returns a seris of dataframes which can be applied to training a \
#machine learning model to predict apppropriate infill values for those points \
#that had missing values from the original sets, indlucing returns of \
#df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
#and df_test_fillfeatures


def createMLinfillsets(df_train, df_test, column, trainNArows, testNArows, \
                       category, columnslist = [], categorylist = []):

  
  
  #create 3 new dataframes for each train column - the train and labels \
  #for rows not needing infill, and the features for rows needing infill \
  #also create a test features column 

  #reminder:
    #for numerical there won't be a new column
    #for binary there won't be a new column
    #for text the new column has a defined name as column+'_missing'

  #note that for text class the labels will be a little more complicated \
  #since will be multi-column

  if category in ['nmbr', 'bxcx', 'bnry', 'text']:
    
    #if this is a single column set (not categorical)
    if categorylist == []:
      
      #first concatinate the NArows True/False designations to df_train & df_test
      df_train = pd.concat([df_train, trainNArows], axis=1)
      df_test = pd.concat([df_test, testNArows], axis=1)

      #create copy of df_train to serve as training set for fill
      df_train_filltrain = df_train.copy()
      #now delete rows coresponding to True
      df_train_filltrain = df_train_filltrain[df_train_filltrain[trainNArows.columns.get_values()[0]] == False]

      #now delete columns = columnslist and the NA labels (orig column+'_NArows') from this df
      df_train_filltrain = df_train_filltrain.drop(columnslist, axis=1)
      df_train_filltrain = df_train_filltrain.drop([trainNArows.columns.get_values()[0]], axis=1)
      
      
      
      #create a copy of df_train[column] for fill train labels
      df_train_filllabel = pd.DataFrame(df_train[column].copy())
      #concatinate with the NArows
      df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
      #drop rows corresponding to True
      df_train_filllabel = df_train_filllabel[df_train_filllabel[trainNArows.columns.get_values()[0]] == False]

      #delete the NArows column
      df_train_filllabel = df_train_filllabel.drop([trainNArows.columns.get_values()[0]], axis=1)

      #create features df_train for rows needing infill
      #create copy of df_train (note it already has NArows included)
      df_train_fillfeatures = df_train.copy()
      #delete rows coresponding to False
      df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[trainNArows.columns.get_values()[0]])]
      #delete columnslist and column+'_NArows'
      df_train_fillfeatures = df_train_fillfeatures.drop(columnslist, axis=1)
      df_train_fillfeatures = df_train_fillfeatures.drop([trainNArows.columns.get_values()[0]], axis=1)
    
      
      #create features df_test for rows needing infill
      #create copy of df_test (note it already has NArows included)
      df_test_fillfeatures = df_test.copy()
      #delete rows coresponding to False
      df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[testNArows.columns.get_values()[0]])]
      #delete column and column+'_NArows'
      df_test_fillfeatures = df_test_fillfeatures.drop(columnslist, axis=1)
      df_test_fillfeatures = df_test_fillfeatures.drop([testNArows.columns.get_values()[0]], axis=1)

      #delete NArows from df_train, df_test
      df_train = df_train.drop([trainNArows.columns.get_values()[0]], axis=1)
      df_test = df_test.drop([testNArows.columns.get_values()[0]], axis=1)
      
    #else if categorylist wasn't empty
    else:
      
      #create a list of columns representing columnslist exlucding elements from
      #categorylist
      noncategorylist = columnslist[:]
      #this removes categorylist elements from noncategorylist
      noncategorylist = list(set(noncategorylist).difference(set(categorylist)))
      
      
      #first concatinate the NArows True/False designations to df_train & df_test
      df_train = pd.concat([df_train, trainNArows], axis=1)
      df_test = pd.concat([df_test, testNArows], axis=1)

      #create copy of df_train to serve as training set for fill
      df_train_filltrain = df_train.copy()
      #now delete rows coresponding to True
      df_train_filltrain = df_train_filltrain[df_train_filltrain[trainNArows.columns.get_values()[0]] == False]

      #now delete columns = columnslist and the NA labels (orig column+'_NArows') from this df
      df_train_filltrain = df_train_filltrain.drop(columnslist, axis=1)
      df_train_filltrain = df_train_filltrain.drop([trainNArows.columns.get_values()[0]], axis=1)

      #create a copy of df_train[columnslist] for fill train labels
      df_train_filllabel = df_train[columnslist].copy()
      #concatinate with the NArows
      df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
      #drop rows corresponding to True
      df_train_filllabel = df_train_filllabel[df_train_filllabel[trainNArows.columns.get_values()[0]] == False]
      
      #now delete columns = noncategorylist from this df
      df_train_filltrain = df_train_filltrain.drop(noncategorylist, axis=1)
      
      #delete the NArows column
      df_train_filllabel = df_train_filllabel.drop([trainNArows.columns.get_values()[0]], axis=1)

      
      #create features df_train for rows needing infill
      #create copy of df_train (note it already has NArows included)
      df_train_fillfeatures = df_train.copy()
      #delete rows coresponding to False
      df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[trainNArows.columns.get_values()[0]])]
      #delete columnslist and column+'_NArows'
      df_train_fillfeatures = df_train_fillfeatures.drop(columnslist, axis=1)
      df_train_fillfeatures = df_train_fillfeatures.drop([trainNArows.columns.get_values()[0]], axis=1)

          
      #create features df_test for rows needing infill
      #create copy of df_test (note it already has NArows included)
      df_test_fillfeatures = df_test.copy()
      #delete rows coresponding to False
      df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[testNArows.columns.get_values()[0]])]
      #delete column and column+'_NArows'
      df_test_fillfeatures = df_test_fillfeatures.drop(columnslist, axis=1)
      df_test_fillfeatures = df_test_fillfeatures.drop([testNArows.columns.get_values()[0]], axis=1)

      #delete NArows from df_train, df_test
      df_train = df_train.drop([trainNArows.columns.get_values()[0]], axis=1)
      df_test = df_test.drop([testNArows.columns.get_values()[0]], axis=1)
      



  if category == 'date':

    #create empty sets for now
    #an extension of this method would be to implement a comparable method \
    #for the time category, based on the columns output from the preprocessing
    df_train_filltrain = pd.DataFrame({'foo' : []}) 
    df_train_filllabel = pd.DataFrame({'foo' : []})
    df_train_fillfeatures = pd.DataFrame({'foo' : []})
    df_test_fillfeatures = pd.DataFrame({'foo' : []})


  return df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures




In [0]:
#insertinfill(df, column, infill, category, NArows, columnslist = [])
#function that takes as input a dataframe, column id, category string of either\
#'nmbr'/'text'/'bnry'/'date', a df column of True/False identifiying row id of\
#rows that will recieve infill, and and a list of columns produced by a text \
#class preprocessor when applicable. Replaces the column cells in rows \
#coresponding to the NArows True values with the values from infill, returns\
#the associated transformed dataframe.


def insertinfill(df, column, infill, category, NArows, columnslist = [], categorylist = []):

  #this is kind of a hack
  #NArows column name uses original column name + _NArows as key
  #current column has original column name + _nmbr or _bxcx at end
  #so we'll drop final 5 characters from column string
  origcolumnname = column[:-5]
  
  if category in ['nmbr', 'bxcx', 'bnry', 'text']:

    #if this is a single column set (not categorical)
    if categorylist == []:
      
      #create new dataframe for infills wherein the infill values are placed in \
      #rows coresponding to NArows True values and rows coresponding to NArows \
      #False values are filled with a 0    

      
      #assign index values to a column
      df['tempindex1'] = df.index

      #concatinate our df with NArows
      df = pd.concat([df, NArows], axis=1)

      #create list of index numbers coresponding to the NArows True values
      infillindex = df.loc[df[origcolumnname+'_NArows']]['tempindex1']
      
      #create a dictionary for use to insert infill using df's index as the key
      infill_dict = dict(zip(infillindex, infill.values))

      #replace 'tempindex1' column with infill in rows where NArows is True
      df['tempindex1'] = np.where(df[origcolumnname+'_NArows'], df['tempindex1'].replace(infill_dict), 'fill')

      #now carry that infill over to the target column for rows where NArows is True
      df[column] = np.where(df[origcolumnname+'_NArows'], df['tempindex1'], df[column])

      #remove the temporary columns from df
      df = df.drop(['tempindex1'], axis=1)
      df = df.drop([origcolumnname+'_NArows'], axis=1)
      
      
    #else if categorylist wasn't empty
    else:
      
      #create new dataframe for infills wherein the infill values are placed in \
      #rows coresponding to NArows True values and rows coresponding to NArows \
      #False values are filled with a 0

      #text infill contains multiple columns for each predicted calssification
      #which were derived from one-hot encoding the original column in preprocessing
      for textcolumnname in categorylist:

        #create newcolumn which will serve as the NArows specific to textcolumnname
        df['textNArows'] = NArows

        df['textNArows'] = df['textNArows'].replace(0, False)
        df['textNArows'] = df['textNArows'].replace(1, True)

        #assign index values to a column
        df['tempindex1'] = df.index


        #create list of index numbers coresponding to the NArows True values
        textinfillindex = pd.DataFrame(df.loc[df['textNArows']]['tempindex1'])
        #reset the index
        textinfillindex = textinfillindex.reset_index()

        #now before we create our infill dicitonaries, we're going to need to
        #create a seperate textinfillindex for each category

        infill['tempindex1'] = textinfillindex['tempindex1']

        #first let's create a copy of this textcolumn's infill column replacing 
        #0/1 with True False (this works because we are one hot encoding)
        infill[textcolumnname + '_bool'] = infill[textcolumnname].astype('bool')

        #we'll use the mask feature to create infillindex which only contains \
        #rows coresponding to the True value in the column we just created

        mask = (infill[[textcolumnname + '_bool']]==True).all(1)
        infillindex = infill[mask]['tempindex1']



        #we're only going to insert the infill to column textcolumnname if we \
        #have infill to insert

        if len(infillindex.values) > 0:

          df.loc[infillindex.values[0], textcolumnname] = 1




        #now we'll delete temporary support columns associated with textcolumnname
        infill = infill.drop([textcolumnname + '_bool'], axis=1)
        infill = infill.drop(['tempindex1'], axis=1)
        df = df.drop(['textNArows'], axis=1)
        df = df.drop(['tempindex1'], axis=1)
      
      

  
  if category == 'date':
    #this spot reserved for future update to incorporate address of datetime\
    #category data
    df = df
  
  
  return df 

# 5) update automunge(.) function for new bxcx category

In [0]:
#automunge(df_train, df_test, labels_column, valpercent=0.20, powertransform = True, \
#MLinfill = True, infilliterate=1, randomseed = 42, excludetransformscolumns = []) \
#Function that when fed a train and test data set automates the process \
#of evaluating each column for determination and applicaiton of appropriate \
#preprocessing. Takes as arguement pandas dataframes of training and test data \
#(mdf_train), (mdf_test), the name of the column from train set containing \
#labels, a string identifying the ID column for train and test, a value for \
#percent of training data to be applied to a validation set, a True'False selector \
#to determine if a power law transoffrmation will be applied to numerical sets, a \
#True/False selector to determine if MLinfill methods will be applied to any missing \
#points, an integer indication how many iterations of infill predfictions to \
#run, a random seed integer, and a list of any stroing column names that are to\
#be excluded from processing. (If MLinfill = False, missing points are addressed \
#with mean for numerical, most common value for binary, new column for one-hot \
#encoding, and mean for datetime). Note that the ML method for datetime data is \
#future extension. Based on an evaluation of columns selectively applies one of \
#four preprocessing functions to each. Shuffles the data and splits the training \
#set into train and validation sets. 
#returns train, trainID, labels, validation, validationID, validationlabels, \
#test, testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test,  \
#postprocess_dict

#Note that this approach assumes that the test data is available at time of training
#For subsequent processing of test data the postmung function can be applied
#with as input the postprocess_dict returned by automunge's address of the train set

#The thinking with the infilliterate approach is that for particularly messy \
#sets the predictinfill method will be influenced by the initial plug value \
#for missing cells, and so multiple iterations of the predictinfill should \
#trend towards better predictions. Initial tests of this iteration did not \
#demonstrate much effect so this probably is not neccesary for common use.

#a word of caution: if you are excluding any columns from processing via \
#excludetransformscolumns list make sure they are already in a suitable state \
#for application of ML (e.g. numerical) otherwise the MLinfill technique will \
#return errors

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

def automunge(df_train, df_test, labels_column, trainID_column = False, \
              testID_column = False, valpercent=0.20, powertransform = True, \
              MLinfill = True, infilliterate=1, randomseed = 42, \
              excludetransformscolumns = []):
  
  #An extension could be to test the input data here for non-dataframe format \
  #(such as csv) to convert it to pandas within the function. 
  
  #my understanding is it is good practice to convert any None values into NaN \
  #so I'll just get that out of the way
  df_train.fillna(value=float('nan'), inplace=True)
  df_test.fillna(value=float('nan'), inplace=True)
  
  #we'll delete any rows from training set missing values in the labels column
  df_train = df_train.dropna(subset=[labels_column])
  
  #extract the ID columns from train and test set
  if trainID_column != False:
    df_trainID = pd.DataFrame(df_train[trainID_column])
    del df_train[trainID_column]
    
  if testID_column != False:
    df_testID = pd.DataFrame(df_test[testID_column])
    del df_test[testID_column]
  
  #extract labels from train set
  #an extension to this function could be to delete the training set rows\
  #where the labels are missing or improperly formatted prior to performing\
  #this step
  df_labels = pd.DataFrame(df_train[labels_column])
  
  #create copy of labels to support the translation dictionary for use after \
  #prediction to convert encoded predictions back to the original label
  df_labels2 = pd.DataFrame(df_labels.copy())
  
  del df_train[labels_column]
  
  
  #confirm consistency of train an test sets
  
  #check number of columns is consistent
  if df_train.shape[1] != df_test.shape[1]:
    print("error, different number of columns in train and test sets")
    print("(This assesment excludes labels and ID columns.)")
    return
  
  #check column headers are consistent (this works independent of order)
  columns_train = set(list(df_train))
  columns_test = set(list(df_test))
  if columns_train != columns_test:
    print("error, different column labels in the train and test set")
    print("(This assesment excludes labels and ID columns.)")
    return

  columns_train = list(df_train)
  columns_test = list(df_test)
  if columns_train != columns_test:
    print("error, different order of column labels in the train and test set")
    print("(This assesment excludes labels and ID columns.)")
    return
  
  #extract column lists again but this time as a list
  columns_train = list(df_train)
  columns_test = list(df_test)

  
  #create an empty dataframe to serve as a store for each column's NArows
  #the column id's for this df will follow convention from NArows of 
  #column+'_NArows' for each column in columns_train
  #these are used in the ML infill methods
  masterNArows_train = pd.DataFrame()
  masterNArows_test = pd.DataFrame()
  
  
  #create an empty dictionary to serve as store for categorical transforms lists
  #of associated columns
  multicolumntransform_dict = {}
  
  #create an empty dictionary to serve as a store of processing variables from \
  #processing that were specific to the train dataset. These can be used for \
  #future processing of a later test set without the need to reprocess the \
  #original train. The dictionary will be populated with an entry for each \
  #column post processing, and will contain a column specific and category \
  #specific (i.e. nmbr, bnry, text, date) set of variable.
  postprocess_dict = {'column_dict' : {}}
  
  
  #For each column, determine appropriate processing function
  #processing function will be based on evaluation of train set
  for column in columns_train:
    
    #re-initialize the column specific dictionary for later insertion into
    #our postprocess_dict
    column_dict = {}
    
    #we're only going to process columns that weren't in our excluded set
    if column not in excludetransformscolumns:
      
      category = evalcategory(df_train, column)
      
      #let's make sure the category is consistent between train and test sets
      category_test = evalcategory(df_test, column)
      
      #for the special case of train category = bxcx and test category = nmbr
      #(meaning there were no negative values in train but there were in test)
      #we'll resolve by reseting the train category to nmbr
      if category == 'bxcx' and category_test == 'nmbr':
        category = 'nmbr'
        
      #one more bxcx special case: if user elects not to apply boxcox transform
      #default to 'nmbr' category instead of 'bxcx'
      if category == 'bxcx' and powertransform == False:
        category = 'nmbr'
        category_test = 'nmbr'
      
      #otherwise if train category != test category return error
      if category != category_test:
        print('error - different category between train and test sets for column ',\
             column)
        
      #here we'll delete any columns that returned a 'null' category
      if category == 'null':
        df_train = df_train.drop([column], axis=1)
        df_test = df_test.drop([column], axis=1)
        
        column_dict = { column + '_null' : {'category' : 'null', \
                                            'origcategory' : 'null', \
                                            'normalization_dict' : {}, \
                                            'origcolumn' : column, \
                                            'columnslist' : [column], \
                                            'categorylist' : [], \
                                            'infillmodel' : False, \
                                            'infillcomplete' : False }}
        
        #now append column_dict onto postprocess_dict
        postprocess_dict['column_dict'].update(column_dict)

      
      #so if we didn't delete the column let's proceed
      else:




        #create NArows (column of True/False where True coresponds to missing data)
        trainNArows = NArows(df_train, column, category)
        testNArows = NArows(df_test, column, category)

        #now append that NArows onto a master NA rows df
        masterNArows_train = pd.concat([masterNArows_train, trainNArows], axis=1)
        masterNArows_test = pd.concat([masterNArows_test, testNArows], axis=1)


        #(now normalize as would normally)



        #for binary class use the majority field for missing plug value
        if category == 'bnry':
          binary_missing_plug = df_train[column].value_counts().index.tolist()[0]


          #apply appropriate processing function to this column based on the result
          df_train, bnrycolumns, categorylist = process_binary_class(df_train, column, binary_missing_plug)
          df_test, _1, _2 = process_binary_class(df_test, column, binary_missing_plug)
          
          bnrynormalization_dict = {'missing' : binary_missing_plug}
          
          #store some values in the column_dict{} for use later in ML infill methods
          for bc in bnrycolumns:

            
            column_dict = { bc : {'category' : 'bnry', \
                                 'origcategory' : 'bnry', \
                                 'normalization_dict' : bnrynormalization_dict, \
                                 'origcolumn' : column, \
                                 'columnslist' : bnrycolumns, \
                                 'categorylist' : categorylist, \
                                 'infillmodel' : False, \
                                 'infillcomplete' : False}}


            #now append column_dict onto postprocess_dict
            postprocess_dict['column_dict'].update(column_dict)
          
          
        if category == 'nmbr':
          df_train, df_test, mean, std, nmbrcolumns, categorylist = \
          process_numerical_class(df_train, df_test, column)
          
          nmbrnormalization_dict = {'mean' : mean, 'std' : std}
          
          #store some values in the nmbr_dict{} for use later in ML infill methods
          for nc in nmbrcolumns:


            column_dict = { nc : {'category' : 'nmbr' , \
                                  'origcategory' : 'nmbr', \
                                  'normalization_dict' : nmbrnormalization_dict, \
                                  'origcolumn' : column, \
                                  'columnslist' : nmbrcolumns, \
                                  'categorylist' : categorylist, \
                                  'infillmodel' : False, \
                                  'infillcomplete' : False}}

            
            #now append column_dict onto postprocess_dict
            postprocess_dict['column_dict'].update(column_dict)
          

        if category == 'bxcx':
          
          df_train, nmbrcolumns, nmbrnormalization_dict, categorylist = \
          process_bxcx_class(df_train, column, bxcx_lmbda = None, \
                             trnsfrm_mean = None, trnsfrm_std = None)
          
          df_test, nmbrcolumns, _1, _2 = \
          process_bxcx_class(df_test, column, bxcx_lmbda = \
                             nmbrnormalization_dict['bxcx_lmbda'], \
                             trnsfrm_mean = nmbrnormalization_dict['trnsfrm_mean'], \
                             trnsfrm_std = nmbrnormalization_dict['trnsfrm_std'])
          
          
          #store some values in the bxcx_dict{} for use later in ML infill methods
          for nc in nmbrcolumns:

          
            #now create our column_dict for bxcx category 
            column_dict = {nc : {'category' : 'bxcx', \
                                 'origcategory' : 'bxcx', \
                                 'normalization_dict' : nmbrnormalization_dict, \
                                 'origcolumn' : column, \
                                 'columnslist' : nmbrcolumns, \
                                 'categorylist' : categorylist, \
                                 'infillmodel' : False, \
                                 'infillcomplete' : False }}
          
            
            #now append column_dict onto postprocess_dict
            postprocess_dict['column_dict'].update(column_dict)
        
        
        if category == 'text':
          df_train, df_test, textcolumns, categorylist = \
          process_text_class(df_train, df_test, column)
          
          #store some values in the text_dict{} for use later in ML infill methods
          for tc in textcolumns:

          
            column_dict = {tc : {'category' : 'text', \
                                 'origcategory' : 'text', \
                                 'normalization_dict' : {}, \
                                 'origcolumn' : column, \
                                 'columnslist' : textcolumns, \
                                 'categorylist' : categorylist, \
                                 'infillmodel' : False, \
                                 'infillcomplete' : False }}

    
    
            #now append column_dict onto postprocess_dict
            postprocess_dict['column_dict'].update(column_dict)
          

        if category == 'date':
          df_train, df_test, datecolumns, timenormalization_dict, categorylist = \
          process_time_class(df_train, df_test, column)

          #store some values in the date_dict{} for use later in ML infill methods

          for dc in datecolumns:

            
            column_dict = {dc :  {'category' : 'date', \
                                  'origcategory' : 'date', \
                                  'normalization_dict' : timenormalization_dict, \
                                  'origcolumn' : column, \
                                  'columnslist' : datecolumns, \
                                  'categorylist' : categorylist, \
                                  'infillmodel' : False, \
                                  'infillcomplete' : False }}
                                  
            

            #now append column_dict onto postprocess_dict
            postprocess_dict['column_dict'].update(column_dict)

  
  #now that we've pre-processed all of the columns, let's run through them again\
  #using ML to derive plug values for the previously missing cells

  
  if MLinfill == True:
    
    
    columns_train_ML = list(df_train)
    columns_test_ML = list(df_test)
    
    
    iteration = 0
    
    while iteration < infilliterate:

        
      for key in postprocess_dict['column_dict']:
        postprocess_dict['column_dict'][key]['infillcomplete'] = False
      
      
      for column in columns_train_ML:


        #we're only going to process columns that weren't in our excluded set
        #or aren't identifiers for NA rows
        if column not in excludetransformscolumns \
        and column[-5:] != '_NArw':


          
          #If column id is found in the text_dict then will require different \
          #type of address since this category won't be found in our \
          #mastercategory_dict and we'll need to apply the ML infill to the \
          #collective group of columns from the associated textcolumns array.

          #if column in text_dict:
          if column[-5:] == '_text':
          

            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:

              #pull this column's textcolumns list
              textcolumns = postprocess_dict['column_dict'][column]['columnslist']
              textcategorylist = postprocess_dict['column_dict'][column]['categorylist']

              category = 'text'

              #now let's apply our functions for ML infill

              #createMLinfillsets
              df_train_filltrain, df_train_filllabel, df_train_fillfeatures, df_test_fillfeatures = \
              createMLinfillsets(df_train, \
                                 df_test, column, \
                                 pd.DataFrame(masterNArows_train[postprocess_dict['column_dict'][column]['origcolumn']+'_NArows']), \
                                 pd.DataFrame(masterNArows_test[postprocess_dict['column_dict'][column]['origcolumn']+'_NArows']), \
                                 category, \
                                 columnslist = textcolumns, \
                                 categorylist = textcategorylist)          


              #predict infill values using defined function predictinfill(.)
              df_traininfill, df_testinfill, model = \
              predictinfill(category, df_train_filltrain, df_train_filllabel, \
                            df_train_fillfeatures, df_test_fillfeatures, \
                            randomseed, columnslist = textcolumns)
              
              #apply the function insertinfill(.) to insert missing value predicitons \
              #to df's associated column
              
              df_train = insertinfill(df_train, column, df_traininfill, category, \
                                      pd.DataFrame(masterNArows_train[postprocess_dict['column_dict'][column]['origcolumn']+'_NArows']), \
                                      columnslist = textcolumns, categorylist = textcategorylist)
              
              #now we'll add our trained text model to the postprocess_dict
              postprocess_dict['column_dict'][column]['infillmodel'] \
              = model
              
              #troubleshooting note: it occurs to me that we're only saving our
              #trained model in the postprocess_dict for one of the text columns
              #not all, however since this will be the first column to be 
              #addressed here and also in the postmunge function (they're in 
              #same order) my expectation is that this will not be an issue and \
              #accidental bonus since we're only saving once results in reduced
              #file size
              
              
              #it's a quirk of the ML models that if we don't train the
              #train set model on any features, that we won't be able to apply
              #the model to predict the test set infill. 
              #For now we'll only use insertilnfill if we had
              #some missing points in the train set, a future extension would be
              #to update our createMLinfillsets and predictinfill to create 
              #some arbitrary features to train the infill on train set for
              #cases where there are NaN values in test set but not in train
              #such that we could insert infill for missing values in test set. 
              if any(x == True for x in masterNArows_train[postprocess_dict['column_dict'][column]['origcolumn']+'_NArows']):
              
                df_test = insertinfill(df_test, column, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[postprocess_dict['column_dict'][column]['origcolumn']+'_NArows']), \
                                       columnslist = textcolumns, categorylist = textcategorylist)

              #now change the infillcomplete marker in the text_dict for each \
              #associated text column
              for textcolumnname in textcolumns:
                postprocess_dict['column_dict'][textcolumnname]['infillcomplete'] = True
          
          
          #If column id found in the bxcx_dict then we'll address with the
          #box-cox methods
          #elif column in bxcx_dict:
          elif column[-5:] == '_bxcx':
            
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
              
              #pull this column's textcolumns list
              #for the record I know this is a list not an array
              nmbrcolumns = postprocess_dict['column_dict'][column]['columnslist']
              nmbrcategorylist = postprocess_dict['column_dict'][column]['categorylist']
              
              #grab the original column name prior to preprocessing
              nmbrorigcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              
              #now get the transformed column name for the two categories of nmbr and bxcx
              column_bxcx = nmbrorigcolumn + '_bxcx'
              category_bxcx = 'bxcx'
              column_nmbr = nmbrorigcolumn + '_nmbr'
              category_nmbr = 'nmbr'
              
              #now let's apply our functions for ML infill
              
              #for the record I'm sure that the conversion of the single column
              #series to a dataframe is counter to the intent of pandas
              #it's probably less memory efficient but it's the current basis of
              #the functions so we're going to maintain that approach for now
              #the revision of these functions to accept pandas series is a
              #possible future extension
              
              #createMLinfillsets
              
              df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
              df_test_fillfeatures = \
              createMLinfillsets(df_train, df_test, column_bxcx, \
                                 pd.DataFrame(masterNArows_train[nmbrorigcolumn+'_NArows']), \
                                 pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                 category_bxcx, columnslist = nmbrcolumns, \
                                 categorylist = nmbrcategorylist)
              
              #predict infill values using defined function predictinfill(.)
              #first we'll address the bxcx transformed column
              df_traininfill_bxcx, df_testinfill_bxcx, model_bxcx = \
                            predictinfill(category_bxcx, df_train_filltrain, \
                            df_train_filllabel, \
                            df_train_fillfeatures, df_test_fillfeatures, \
                            randomseed, columnslist = nmbrcolumns)

              
              #apply the function insertinfill(.) to insert missing value predicitons \
              #to df's associated column
              #first we'll address the bxcx transformed column
              df_train = insertinfill(df_train, column_bxcx, df_traininfill_bxcx, \
                                      category_bxcx, \
                                      pd.DataFrame(masterNArows_train[nmbrorigcolumn+'_NArows']), \
                                      columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
  
  
  
  
              #now we'll address the nmbr column
              
              df_traininfill_nmbr, df_testinfill_nmbr, model_nmbr = \
              predictinfill(category_nmbr, df_train_filltrain, \
                            df_train_filllabel, \
                            df_train_fillfeatures, df_test_fillfeatures, \
                            randomseed, columnslist = nmbrcolumns)


              
              #then we'll address the nmbr transformed column
              df_train = insertinfill(df_train, column_nmbr, df_traininfill_nmbr, \
                                      category_bxcx, \
                                      pd.DataFrame(masterNArows_train[nmbrorigcolumn+'_NArows']), \
                                      columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
              
              
              #now we'll add our trained text model to the postprocess_dict
              postprocess_dict['column_dict'][column[:-5] + '_bxcx']['infillmodel'] \
              = model_bxcx
              
              postprocess_dict['column_dict'][column[:-5] + '_nmbr']['infillmodel'] \
              = model_nmbr
              
              
              #only use insertilnfill if we had some missing points in the train set
              if any(x == True for x in masterNArows_train[nmbrorigcolumn + '_NArows']):
              
                df_test = insertinfill(df_test, column_bxcx, df_testinfill_bxcx, \
                                       category_bxcx, \
                                       pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                       columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
                
                df_test = insertinfill(df_test, column_nmbr, df_testinfill_nmbr, \
                                       category_bxcx, \
                                       pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                       columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
                

              #now change the infillcomplete marker in the text_dict for each \
              #associated text column
              for nmbrcolumnname in nmbrcolumns:
                postprocess_dict['column_dict'][nmbrcolumnname]['infillcomplete'] = True
              
              

          #If column id is found in the date_dict then will require different \
          #type of address since this category won't be found in our \
          #mastercategory_dict and we'll need to apply the ML infill to the \
          #collective group of columns from the associated datecolumns array. \
          #The development of this address for date columns is a future extension.
          #elif column in date_dict:
          elif column[-5:] == '_date':

            #this section to be a future extension.
            pass


          #elif column in bnry_dict:
          elif column[-5:] == '_bnry':
            
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
              
              #pull this column's bnrycolumns list
              bnrycolumns = postprocess_dict['column_dict'][column]['columnslist']
              bnrycategorylist = postprocess_dict['column_dict'][column]['categorylist']

              category = 'bnry'
              
              #grab the original column name prior to preprocessing
              bnryorigcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              
              #now get the transformed column name for the row needing infill
              bnryinfillcolumn = bnryorigcolumn+'_bnry'
              
              #now let's apply our functions for ML infill
              #createMLinfillsets
              
              df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
              df_test_fillfeatures = \
              createMLinfillsets(df_train, df_test, bnryinfillcolumn, \
                                 pd.DataFrame(masterNArows_train[bnryorigcolumn + '_NArows']), \
                                 pd.DataFrame(masterNArows_test[bnryorigcolumn + '_NArows']), category, \
                                 columnslist = bnrycolumns, categorylist = bnrycategorylist)
              
              #predict infill values using defined function predictinfill(.)
              df_traininfill, df_testinfill, model = \
              predictinfill(category, df_train_filltrain, df_train_filllabel, \
                            df_train_fillfeatures, df_test_fillfeatures, \
                            randomseed, columnslist = bnrycolumns)
              
              #apply the function insertinfill(.) to insert missing value predicitons \
              #to df's associated column
              df_train = insertinfill(df_train, bnryinfillcolumn, df_traininfill, category, \
                                      pd.DataFrame(masterNArows_train[bnryorigcolumn + '_NArows']), \
                                      columnslist = bnrycolumns, categorylist = bnrycategorylist)
              
              #(only insert infill to test set if we had NA rows in train set)
              #note comments above in text class for potential future expansion
              if any(x == True for x in masterNArows_train[bnryorigcolumn+'_NArows']):
                
                df_test = insertinfill(df_test, bnryinfillcolumn, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[bnryorigcolumn + '_NArows']), \
                                       columnslist = bnrycolumns, categorylist = bnrycategorylist)
              
              #now we'll add our trained bnry model to the postprocess_dict
              postprocess_dict['column_dict'][bnryinfillcolumn]['infillmodel'] = model
          
          #elif column in nmbr_dict:
          elif column[-5:] == '_nmbr':
            
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
              
              #pull this column's nmbrcolumns list
              nmbrcolumns = postprocess_dict['column_dict'][column]['columnslist']
              nmbrcategorylist = postprocess_dict['column_dict'][column]['categorylist']
              
              category = 'nmbr'
              
              #grab the original column name prior to preprocessing
              nmbrorigcolumn = postprocess_dict['column_dict'][column]['origcolumn']

              #now get the transformed column name for the row needing infill
              nmbrinfillcolumn = nmbrorigcolumn+'_nmbr'
              
              #create MLinfill sets using defined function createMLinfillsets(.)
              df_train_filltrain, df_train_filllabel, df_train_fillfeatures, \
              df_test_fillfeatures = \
              createMLinfillsets(df_train, df_test, nmbrinfillcolumn, \
                                 pd.DataFrame(masterNArows_train[nmbrorigcolumn+'_NArows']), \
                                 pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                 category, columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
              
              #predict infill values using defined function predictinfill(.)
              df_traininfill, df_testinfill, model = \
              predictinfill(category, df_train_filltrain, df_train_filllabel, \
                            df_train_fillfeatures, df_test_fillfeatures, \
                            randomseed, columnslist = nmbrcolumns)
              
              #apply the function insertinfill(.) to insert missing value predicitons \
              #to df's associated column
              df_train = insertinfill(df_train, nmbrinfillcolumn, df_traininfill, category, \
                                      pd.DataFrame(masterNArows_train[nmbrorigcolumn+'_NArows']), \
                                      columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
                            
              #(only insert infill to test set if we had NA rows in train set)
              #note comments above in text class for potential future expansion
              if any(x == True for x in masterNArows_train[nmbrorigcolumn+'_NArows']):
                
                df_test = insertinfill(df_test, nmbrinfillcolumn, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                       columnslist = nmbrcolumns, categorylist = nmbrcategorylist)
              
              #now we'll add our trained nmbr model to the postprocess_dict
              postprocess_dict['column_dict'][nmbrinfillcolumn]['infillmodel'] = model
          
          

            
      iteration += 1

  
  #determine labels category and apply appropriate function
  labelscategory = evalcategory(df_labels, labels_column)
  
  #empty dummy labels "test" df for our preprocessing functions
  labelsdummy = pd.DataFrame()
  
  #initialize a dictionary to serve as the store between labels and their \
  #associated encoding
  labelsencoding_dict = {labelscategory:{}}
  
  #apply appropriate processing function to this column based on the result
  if labelscategory == 'bnry':
    labels_binary_missing_plug = df_labels[labels_column].value_counts().index.tolist()[0]
    df_labels, _1, _2 = process_binary_class(df_labels, labels_column, labels_binary_missing_plug)
    
    #here we'll populate the dictionery pairing values from the encoded labels \
    #column with the original value for transformation post prediciton
    
    
    i = 0
    
    for row in df_labels.iterrows():
      if row[1][0] in labelsencoding_dict[labelscategory].keys():
          i += 1
      else:
          labelsencoding_dict[labelscategory].update({row[1][0] : df_labels2.iloc[i][0]})
          i += 1

      
  if labelscategory == 'nmbr':
    
    #if labels category is 'nmbr' we won't apply any further processing to the \
    #column as my experience with linear regression methods is that this is not\
    #required. Further processing of numerical labels would need to be addressed\
    #by returning mean and std from the process_numerical_class method so as to\
    #potentially store in our labelsencoding_dict
    pass
    
    
  #it occurs to me there might be an argument for preferring a single numerical \
  #classifier for labels to keep this to a single column, if so scikitlearn's \
  #LabelEcncoder could be used here, will assume that onehot encoding is acceptable
  if labelscategory == 'text':
    
    df_labels, labelsdummy, _1, _2 = \
    process_text_class(df_labels, labelsdummy, labels_column)
  
    i = 0
    
    for row in df_labels2.iterrows():
      if row[1][0] in labelsencoding_dict[labelscategory].keys():
          i += 1
      else:
          labelsencoding_dict[labelscategory].\
          update({row[1][0] : labels_column+'_'+row[1][0]})
          i += 1
    
  
  #great the data is processed now let's do a few moore global training preps
  
  
  #here's a list of final column names saving here since the translation to \
  #numpy arrays scrubs the column names
  finalcolumns_train = list(df_train)
  finalcolumns_test = list(df_test)
  
  
  #convert all of our dataframes to numpy arrays (train, test, labels, and ID)
  #    df_trainID, df_testID
  np_train = df_train.values
  np_test = df_test.values
  np_labels = df_labels.values
  
  if trainID_column != False:
    np_trainID = df_trainID.values
  if testID_column != False:
    np_testID = df_testID.values
  
  
  #set randomness seed number
  answer = randomseed
  #a reasonable extension would be to tie this in with randomness seed for \
  #ML infill methods calls to scikit learn
  
  #shuffle training set and labels
  np_train = shuffle(np_train, random_state = answer)
  np_labels = shuffle(np_labels, random_state = answer)
  
  if trainID_column != False:
    np_trainID = shuffle(np_trainID, random_state = answer)
  
  
  #split validation sets from training and labels
  train, validation, labels, validationlabels = \
  train_test_split(np_train, np_labels, test_size=valpercent, shuffle = False)
  
  if trainID_column != False:
    trainID, validationID = \
    train_test_split(np_trainID, test_size=valpercent, shuffle = False)
  else:
    trainID = []
    validationID = []
  if testID_column != False:
    testID = np_testID
  else:
    testID = []
  
  test = np_test
  

  
  postprocess_dict.update({'origtraincolumns' : columns_train, \
                           'finalcolumns_train' : finalcolumns_train, \
                           'testID_column' : testID_column, \
                           'MLinfill' : MLinfill, \
                           'infilliterate' : infilliterate, \
                           'randomseed' : randomseed, \
                           'excludetransformscolumns' : excludetransformscolumns,\
                           'labelsencoding_dict' : labelsencoding_dict, \
                           'automungeversion' : '1.1' })



  
  
  #a reasonable extension would be to perform some validation functions on the\
  #sets here (or also prior to transofrm to numpuy arrays) and confirm things \
  #like consistency between format of columns and data between our train and \
  #test sets and if any issues return a coresponding error message to alert user
  
  
  return train, trainID, labels, validation, validationID, validationlabels, \
  test, testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test,  \
  postprocess_dict


Note that a future extension will be to create some standardized functions recreatelabels(.) for post prediction transforms of predicitons from numerical to string values if applicable.

# 6) update postprocess functions for test set

In [0]:
# #Here is a summary of the postprocess_dict structure from automunge:



# postprocess_dict.update({'origtraincolumns' : columns_train, \
#                          'finalcolumns_train' : finalcolumns_train, \
#                          'testID_column' : testID_column, \
#                          'MLinfill' : MLinfill, \
#                          'infilliterate' : infilliterate, \
#                          'randomseed' : randomseed, \
#                          'excludetransformscolumns' : excludetransformscolumns,\
#                          'labelsencoding_dict' : labelsencoding_dict, \
#                          'automungeversion' : '1.1', 
#                          'column_dict' : {}})

# (example of bnry)
# column_dict = { bc : {'category' : 'bnry', \
#                      'origcategory' : 'bnry', \
#                      'normalization_dict' : bnrynormalization_dict, \
#                      'origcolumn' : column, \
#                      'columnslist' : bnrycolumns, \
#                      'categorylist' : categorylist, \
#                      'infillmodel' : False, \
#                      'infillcomplete' : False}}

# nmbr
# nmbrnormalization_dict = {'mean' : mean, 'std' : std}

# bxcx
# nmbrnormalization_dict = {'trnsfrm_mean' : mean, 'trnsfrm_std' : std, \
#                           'bxcx_lmbda' : bxcx_lmbda}



# bnrynormalization_dict = {'missing' : binary_missing_plug}



# timenormalization_dict = {'meanyear' : meanyear, 'meanmonth' : meanmonth, \
#                           'meanday' : meanday, 'meanhour' : meanhour, \
#                           'meanminute' : meanminute, 'meansecond' : meansecond,\
#                           'stdyear' : stdyear, 'stdmonth' : stdmonth, \
#                           'stdday' : stdday, 'stdhour' : stdhour, \
#                           'stdminute' : stdminute, 'stdsecond' : stdsecond}




In [0]:
#postprocess_numerical_class(mdf_test, column, mean, std)
#function to normalize data to mean of 0 and standard deviation of 1 from training distribution
#takes as arguement pandas dataframe of training and test data (mdf_train), (mdf_test)\
#and the name of the column string ('column'), and the mean and std from the train set \
#replaces missing or improperly formatted data with mean of remaining values
#replaces original specified column in dataframe
#returns transformed dataframe

#expect this approach works better when the numerical distribution is thin tailed
#if only have training but not test data handy, use same training data for both dataframe inputs

#imports
from pandas import Series
from sklearn import preprocessing

def postprocess_numerical_class(mdf_test, column, mean, std):
     
  
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_nmbr_test = NArows(mdf_test, column, 'nmbr')
  mdf_test[column + '_NArw'] = NArows_nmbr_test.copy()
  del NArows_nmbr_test
  
  #change NArows data type to 8-bit (1 byte) integers for memory savings
  mdf_test[column + '_NArw'] = mdf_test[column + '_NArw'].astype(np.int8)
  
  #convert all values to either numeric or NaN
  mdf_test[column] = pd.to_numeric(mdf_test[column], errors='coerce')

  #get mean of training data
  mean = mean  

  #replace missing data with training set mean
  mdf_test[column] = mdf_test[column].fillna(mean)

  #subtract mean from column
  mdf_test[column] = mdf_test[column] - mean

  #get standard deviation of training data
  std = std

  #divide column values by std
  mdf_test[column] = mdf_test[column] / std
  
  #change column name to column + '_nmbr'
  mdf_test[column + '_nmbr'] = mdf_test[column].copy()
  del mdf_test[column]


  return mdf_test




#postprocess_text_class(mdf_test, column, textcolumns)
#process column with text classifications
#takes as arguement pandas dataframe containing test data  
#()mdf_test), and the name of the column string ('column'), and an array of
#the associated transformed column s from the train set (textcolumns)

#note this aligns formatting of transformed columns to the original train set
#fromt he original treatment with automunge

#deletes the original column from master dataframe and
#replaces with onehot encodings
#with columns named after column_ + text classifications
#missing data replaced with category label 'missing'+column
#any categories missing from the training set removed from test set
#any category present in training but missing from test set given a column of zeros for consistent formatting
#ensures order of all new columns consistent between both sets
#returns two transformed dataframe (mdf_train, mdf_test) \
#and a list of the new column names (textcolumns)

#note it is kind of a hack here to create a column for missing values with \
#two underscores (__) in the column name to ensure appropriate order for cases\
#where NaN present in test data but not train data, if a category starts with|
#an underscore such that it preceeds '__missing' alphabetically in this scenario\
#this might create error due to different order of columns, address of this \
#potential issue will be a future extension


from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

def postprocess_text_class(mdf_test, column, textcolumns):

  #replace NA with a dummy variable
  mdf_test[column] = mdf_test[column].fillna('_missing')


  #extract categories for column labels
  #note that .unique() extracts the labels as a numpy array
  
  #we'll get the category names from the textcolumns array by stripping the \
  #prefixes of column name + '_'
  prefixlength = len(column)+1
  labels_train = textcolumns[:]
  for textcolumn in labels_train:
    textcolumn = textcolumn[prefixlength :]
  #labels_train.sort(axis=0)
  labels_train.sort()
  labels_test = mdf_test[column].unique()
  labels_test.sort(axis=0)

  #transform text classifications to numerical id
  encoder = LabelEncoder()
#   cat_train = mdf_train[column]
#   cat_train_encoded = encoder.fit_transform(cat_train)

  cat_test = mdf_test[column]
  cat_test_encoded = encoder.fit_transform(cat_test)


  #apply onehotencoding
  onehotencoder = OneHotEncoder()
#   cat_train_1hot = onehotencoder.fit_transform(cat_train_encoded.reshape(-1,1))
  cat_test_1hot = onehotencoder.fit_transform(cat_test_encoded.reshape(-1,1))

  #append column header name to each category listing
  #note the iteration is over a numpy array hence the [...] approach  
#   labels_train[...] = column + '_' + labels_train[...]
  labels_test[...] = column + '_' + labels_test[...]


  #convert sparse array to pandas dataframe with column labels
#   df_train_cat = pd.DataFrame(cat_train_1hot.toarray(), columns=labels_train)
  df_test_cat = pd.DataFrame(cat_test_1hot.toarray(), columns=labels_test)
  
#   #add a missing column to train if it's not present
#   if column + '__missing' not in df_train_cat.columns:
#     missingcolumn = pd.DataFrame(0, index=np.arange(df_train_cat.shape[0]), columns=[column+'__missing'])
#     df_train_cat = pd.concat([missingcolumn, df_train_cat], axis=1)


  #Get missing columns in test set that are present in training set
  missing_cols = set( textcolumns ) - set( df_test_cat.columns )
  
  #Add a missing column in test set with default value equal to 0
  for c in missing_cols:
      df_test_cat[c] = 0
  
  #Ensure the order of column in the test set is in the same order than in train set
  #Note this also removes categories in test set that aren't present in training set
  df_test_cat = df_test_cat[textcolumns]


  #concatinate the sparse set with the rest of our training data
#   mdf_train = pd.concat([df_train_cat, mdf_train], axis=1)
  mdf_test = pd.concat([df_test_cat, mdf_test], axis=1)


  #delete original column from training data
#   del mdf_train[column]    
  del mdf_test[column]
  
#   #create output of a list of the created column names
#   labels_train = list(df_train_cat)
#   textcolumns = labels_train
  
  #change data types to 8-bit (1 byte) integers for memory savings
  for textcolumn in textcolumns:
    mdf_test[textcolumn] = mdf_test[textcolumn].astype(np.int8)
  

  return mdf_test



#postprocess_time_class(mdf_test, column, datecolumns, timenormalization_dict)
#postprocess test column with of date category
#takes as arguement pandas dataframe containing test data 
#(mdf_test), the name of the column string ('column'), and the timenormalization_dict 
#from the original application of automunge to the associated date column from train set

#deletes the original column from master dataframe and
#replaces with distinct columns for year, month, day, hour, minute, second
#each normalized to the mean and std from original train set, 
#with missing values plugged with the mean
#with columns named after column_ + time category
#returns two transformed dataframe (mdf_train, mdf_test)

import datetime as dt

def postprocess_time_class(mdf_test, column, datecolumns, timenormalization_dict):
  
  #add a second column with boolean expression indicating a missing cell
  #(using NArows(.) function defined below, column name will be column+'_NArows')
  NArows_nmbr_test = NArows(mdf_test, column, 'nmbr')
  mdf_test[column + '_NArw'] = NArows_nmbr_test.copy()
  del NArows_nmbr_test
  
  #change NArows data type to 8-bit (1 byte) integers for memory savings
  mdf_test[column + '_NArw'] = mdf_test[column + '_NArw'].astype(np.int8)
  
  
  #apply pd.to_datetime to column, note that the errors = 'coerce' needed for messy data
#   mdf_train[column] = pd.to_datetime(mdf_train[column], errors = 'coerce')
  mdf_test[column] = pd.to_datetime(mdf_test[column], errors = 'coerce')
  
  #mdf_train[column].replace(-np.Inf, np.nan)
  #mdf_test[column].replace(-np.Inf, np.nan)
  
  #get mean of various categories of datetime objects to use to plug in missing cells
#   meanyear = mdf_train[column].dt.year.mean()    
#   meanmonth = mdf_train[column].dt.month.mean()
#   meanday = mdf_train[column].dt.day.mean()
#   meanhour = mdf_train[column].dt.hour.mean()
#   meanminute = mdf_train[column].dt.minute.mean()
#   meansecond = mdf_train[column].dt.second.mean()
  
  meanyear = timenormalization_dict['meanyear']
  meanmonth = timenormalization_dict['meanmonth']
  meanday = timenormalization_dict['meanday']
  meanhour = timenormalization_dict['meanhour']
  meanminute = timenormalization_dict['meanminute']
  meansecond = timenormalization_dict['meansecond']
  

  #get standard deviation of training data
#   stdyear = mdf_train[column].dt.year.std()  
#   stdmonth = mdf_train[column].dt.month.std()
#   stdday = mdf_train[column].dt.day.std()
#   stdhour = mdf_train[column].dt.hour.std()
#   stdminute = mdf_train[column].dt.minute.std()
#   stdsecond = mdf_train[column].dt.second.std()
  
  stdyear = timenormalization_dict['stdyear']
  stdmonth = timenormalization_dict['stdmonth']
  stdday = timenormalization_dict['stdday']
  stdhour = timenormalization_dict['stdhour']
  stdminute = timenormalization_dict['stdminute']
  stdsecond = timenormalization_dict['stdsecond']
  
  
#   #create new columns for each category in train set
#   mdf_train[column + '_year'] = mdf_train[column].dt.year
#   mdf_train[column + '_month'] = mdf_train[column].dt.month
#   mdf_train[column + '_day'] = mdf_train[column].dt.day
#   mdf_train[column + '_hour'] = mdf_train[column].dt.hour
#   mdf_train[column + '_minute'] = mdf_train[column].dt.minute
#   mdf_train[column + '_second'] = mdf_train[column].dt.second
  
  #create new columns for each category in test set
  mdf_test[column + '_year'] = mdf_test[column].dt.year
  mdf_test[column + '_month'] = mdf_test[column].dt.month
  mdf_test[column + '_day'] = mdf_test[column].dt.day
  mdf_test[column + '_hour'] = mdf_test[column].dt.hour
  mdf_test[column + '_minute'] = mdf_test[column].dt.minute 
  mdf_test[column + '_second'] = mdf_test[column].dt.second
  

#   #replace missing data with training set mean
#   mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(meanyear)
#   mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(meanmonth)
#   mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(meanday)
#   mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(meanhour)
#   mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(meanminute)
#   mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(meansecond)
  
  #do same for test set (replace missing data with training set mean)
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(meanyear)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(meanmonth)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(meanday)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(meanhour)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(meanminute)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(meansecond)
  
  #subtract mean from column for both train and test
#   mdf_train[column + '_year'] = mdf_train[column + '_year'] - meanyear
#   mdf_train[column + '_month'] = mdf_train[column + '_month'] - meanmonth
#   mdf_train[column + '_day'] = mdf_train[column + '_day'] - meanday
#   mdf_train[column + '_hour'] = mdf_train[column + '_hour'] - meanhour
#   mdf_train[column + '_minute'] = mdf_train[column + '_minute'] - meanminute
#   mdf_train[column + '_second'] = mdf_train[column + '_second'] - meansecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] - meanyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] - meanmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] - meanday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] - meanhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] - meanminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] - meansecond
  
  
  #divide column values by std for both training and test data
#   mdf_train[column + '_year'] = mdf_train[column + '_year'] / stdyear
#   mdf_train[column + '_month'] = mdf_train[column + '_month'] / stdmonth
#   mdf_train[column + '_day'] = mdf_train[column + '_day'] / stdday
#   mdf_train[column + '_hour'] = mdf_train[column + '_hour'] / stdhour
#   mdf_train[column + '_minute'] = mdf_train[column + '_minute'] / stdminute
#   mdf_train[column + '_second'] = mdf_train[column + '_second'] / stdsecond
  
  mdf_test[column + '_year'] = mdf_test[column + '_year'] / stdyear
  mdf_test[column + '_month'] = mdf_test[column + '_month'] / stdmonth
  mdf_test[column + '_day'] = mdf_test[column + '_day'] / stdday
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'] / stdhour
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'] / stdminute
  mdf_test[column + '_second'] = mdf_test[column + '_second'] / stdsecond
  
  
  #now replace NaN with 0
#   mdf_train[column + '_year'] = mdf_train[column + '_year'].fillna(0)
#   mdf_train[column + '_month'] = mdf_train[column + '_month'].fillna(0)
#   mdf_train[column + '_day'] = mdf_train[column + '_day'].fillna(0)
#   mdf_train[column + '_hour'] = mdf_train[column + '_hour'].fillna(0)
#   mdf_train[column + '_minute'] = mdf_train[column + '_minute'].fillna(0)
#   mdf_train[column + '_second'] = mdf_train[column + '_second'].fillna(0)
  
  #do same for test set
  mdf_test[column + '_year'] = mdf_test[column + '_year'].fillna(0)
  mdf_test[column + '_month'] = mdf_test[column + '_month'].fillna(0)
  mdf_test[column + '_day'] = mdf_test[column + '_day'].fillna(0)
  mdf_test[column + '_hour'] = mdf_test[column + '_hour'].fillna(0)
  mdf_test[column + '_minute'] = mdf_test[column + '_minute'].fillna(0)
  mdf_test[column + '_second'] = mdf_test[column + '_second'].fillna(0)
  
    
#   #output of a list of the created column names
#   datecolumns = [column + '_year', column + '_month', column + '_day', \
#                 column + '_hour', column + '_minute', column + '_second']
  
  #this is to address an issue I found when parsing columns with only time no date
  #which returned -inf vlaues, so if an issue will just delete the associated 
  #column along with the entry in datecolumns
#   checkyear = np.isinf(mdf_train.iloc[0][column + '_year'])
#   if checkyear:
#     del mdf_train[column + '_year']
#     if column + '_year' in mdf_test.columns:
#       del mdf_test[column + '_year']

#   checkmonth = np.isinf(mdf_train.iloc[0][column + '_month'])
#   if checkmonth:
#     del mdf_train[column + '_month']
#     if column + '_month' in mdf_test.columns:
#       del mdf_test[column + '_month']

#   checkday = np.isinf(mdf_train.iloc[0][column + '_day'])
#   if checkmonth:
#     del mdf_train[column + '_day']
#     if column + '_day' in mdf_test.columns:
#       del mdf_test[column + '_day']

  #instead we'll just delete a column from test set if not found in train set
  if column + '_year' not in datecolumns:
    del mdf_test[column + '_year']
#     datecolumns.remove(column + '_year')
  if column + '_month' not in datecolumns:
    del mdf_test[column + '_month'] 
#     datecolumns.remove(column + '_month')
  if column + '_day' not in datecolumns:
    del mdf_test[column + '_day']  
#     datecolumns.remove(column + '_day')
  if column + '_hour' not in datecolumns:
    del mdf_test[column + '_hour']
#     datecolumns.remove(column + '_hour')
  if column + '_minute' not in datecolumns:
    del mdf_test[column + '_minute'] 
#     datecolumns.remove(column + '_minute')
  if column + '_second' not in datecolumns:
    del mdf_test[column + '_second'] 
#     datecolumns.remove(column + '_second')
  
  
  #delete original column from training data
  if column in mdf_test.columns:
    del mdf_test[column]  

  
#   #output a dictionary of the associated column mean and std
  
#   timenormalization_dict = {'meanyear' : meanyear, 'meanmonth' : meanmonth, \
#                             'meanday' : meanday, 'meanhour' : meanhour, \
#                             'meanminute' : meanminute, 'meansecond' : meansecond,\
#                             'stdyear' : stdyear, 'stdmonth' : stdmonth, \
#                             'stdday' : stdday, 'stdhour' : stdhour, \
#                             'stdminute' : stdminute, 'stdsecond' : stdsecond}
  
  
  return mdf_test


# 7) update postprocess ML infill functions for test set

In [0]:
#createpostMLinfillsets(df_test, column, testNArows, category, \
#columnslist = []) function that when fed dataframe of
#test set, column id, df of True/False corresponding to rows from original \
#sets with missing values, a string category of 'text', 'date', 'nmbr', or \
#'bnry', and a list of column id's for the text category if applicable. The \
#function returns a series of dataframes which can be applied to apply a \
#machine learning model previously trained on our train set as part of the 
#original automunge application to predict apppropriate infill values for those\
#points that had missing values from the original sets, returning the dataframe\
#df_test_fillfeatures

def createpostMLinfillsets(df_test, column, testNArows, category, \
                           columnslist = [], categorylist = []):

  if category in ['nmbr', 'bxcx', 'bnry', 'text']:
    
    #if this is a single column set (not categorical)
    if categorylist == []:
      
      #first concatinate the NArows True/False designations to df_train & df_test
#       df_train = pd.concat([df_train, trainNArows], axis=1)
      df_test = pd.concat([df_test, testNArows], axis=1)

#       #create copy of df_train to serve as training set for fill
#       df_train_filltrain = df_train.copy()
#       #now delete rows coresponding to True
#       df_train_filltrain = df_train_filltrain[df_train_filltrain[trainNArows.columns.get_values()[0]] == False]

#       #now delete columns = columnslist and the NA labels (orig column+'_NArows') from this df
#       df_train_filltrain = df_train_filltrain.drop(columnslist, axis=1)
#       df_train_filltrain = df_train_filltrain.drop([trainNArows.columns.get_values()[0]], axis=1)
      
      
#       #create a copy of df_train[column] for fill train labels
#       df_train_filllabel = pd.DataFrame(df_train[column].copy())
#       #concatinate with the NArows
#       df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
#       #drop rows corresponding to True
#       df_train_filllabel = df_train_filllabel[df_train_filllabel[trainNArows.columns.get_values()[0]] == False]

#       #delete the NArows column
#       df_train_filllabel = df_train_filllabel.drop([trainNArows.columns.get_values()[0]], axis=1)

#       #create features df_train for rows needing infill
#       #create copy of df_train (note it already has NArows included)
#       df_train_fillfeatures = df_train.copy()
#       #delete rows coresponding to False
#       df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[trainNArows.columns.get_values()[0]])]
#       #delete columnslist and column+'_NArows'
#       df_train_fillfeatures = df_train_fillfeatures.drop(columnslist, axis=1)
#       df_train_fillfeatures = df_train_fillfeatures.drop([trainNArows.columns.get_values()[0]], axis=1)
  
  
      #create features df_test for rows needing infill
      #create copy of df_test (note it already has NArows included)
      df_test_fillfeatures = df_test.copy()
      #delete rows coresponding to False
      df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[testNArows.columns.get_values()[0]])]
      #delete column and column+'_NArows'
      df_test_fillfeatures = df_test_fillfeatures.drop(columnslist, axis=1)
      df_test_fillfeatures = df_test_fillfeatures.drop([testNArows.columns.get_values()[0]], axis=1)

      #delete NArows from df_train, df_test
#       df_train = df_train.drop([trainNArows.columns.get_values()[0]], axis=1)
      df_test = df_test.drop([testNArows.columns.get_values()[0]], axis=1)
  
    #else if categorylist wasn't empty
    else:
      
      #create a list of columns representing columnslist exlucding elements from
      #categorylist
      noncategorylist = columnslist[:]
      #this removes categorylist elements from noncategorylist
      noncategorylist = list(set(noncategorylist).difference(set(categorylist)))
      
      
      #first concatinate the NArows True/False designations to df_train & df_test
#       df_train = pd.concat([df_train, trainNArows], axis=1)
      df_test = pd.concat([df_test, testNArows], axis=1)

#       #create copy of df_train to serve as training set for fill
#       df_train_filltrain = df_train.copy()
#       #now delete rows coresponding to True
#       df_train_filltrain = df_train_filltrain[df_train_filltrain[trainNArows.columns.get_values()[0]] == False]

#       #now delete columns = columnslist and the NA labels (orig column+'_NArows') from this df
#       df_train_filltrain = df_train_filltrain.drop(columnslist, axis=1)
#       df_train_filltrain = df_train_filltrain.drop([trainNArows.columns.get_values()[0]], axis=1)
      
      
#       #create a copy of df_train[columnslist] for fill train labels
#       df_train_filllabel = df_train[columnslist].copy()
#       #concatinate with the NArows
#       df_train_filllabel = pd.concat([df_train_filllabel, trainNArows], axis=1)
#       #drop rows corresponding to True
#       df_train_filllabel = df_train_filllabel[df_train_filllabel[trainNArows.columns.get_values()[0]] == False]
      
#       #now delete columns = noncategorylist from this df
#       df_train_filltrain = df_train_filltrain.drop(noncategorylist, axis=1)
      
#       #delete the NArows column
#       df_train_filllabel = df_train_filllabel.drop([trainNArows.columns.get_values()[0]], axis=1)
      
      
#       #create features df_train for rows needing infill
#       #create copy of df_train (note it already has NArows included)
#       df_train_fillfeatures = df_train.copy()
#       #delete rows coresponding to False
#       df_train_fillfeatures = df_train_fillfeatures[(df_train_fillfeatures[trainNArows.columns.get_values()[0]])]
#       #delete columnslist and column+'_NArows'
#       df_train_fillfeatures = df_train_fillfeatures.drop(columnslist, axis=1)
#       df_train_fillfeatures = df_train_fillfeatures.drop([trainNArows.columns.get_values()[0]], axis=1)
      
      
      #create features df_test for rows needing infill
      #create copy of df_test (note it already has NArows included)
      df_test_fillfeatures = df_test.copy()
      #delete rows coresponding to False
      df_test_fillfeatures = df_test_fillfeatures[(df_test_fillfeatures[testNArows.columns.get_values()[0]])]
      #delete column and column+'_NArows'
      df_test_fillfeatures = df_test_fillfeatures.drop(columnslist, axis=1)
      df_test_fillfeatures = df_test_fillfeatures.drop([testNArows.columns.get_values()[0]], axis=1)

      #delete NArows from df_train, df_test
#       df_train = df_train.drop([trainNArows.columns.get_values()[0]], axis=1)
      df_test = df_test.drop([testNArows.columns.get_values()[0]], axis=1)
      
  if category == 'date':
    
    #create empty sets for now
    #an extension of this method would be to implement a comparable method \
    #for the time category, based on the columns output from the preprocessing
#     df_train_filltrain = pd.DataFrame({'foo' : []}) 
#     df_train_filllabel = pd.DataFrame({'foo' : []})
#     df_train_fillfeatures = pd.DataFrame({'foo' : []})
    df_test_fillfeatures = pd.DataFrame({'foo' : []})
  
  
  return df_test_fillfeatures
  

In [0]:

#predictpostinfill(category, model, df_test_fillfeatures, \
#columnslist = []), function that takes as input \
#a category string, a model trained as part of automunge on the coresponding \
#column from the train set, the output of createpostMLinfillsets(.), a seed \
#for randomness, and a list of columns \
#produced by a text class preprocessor when applicable and returns \
#predicted infills for the test feature sets as df_testinfill based on \
#derivations using scikit-learn, with the lenth of \
#infill consistent with the number of True values from NArows


#imports for numerical class training
#from sklearn.linear_model import LinearRegression
#from sklearn.linear_model import PassiveAggressiveRegressor
#from sklearn.linear_model import Ridge
#from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR

#imports for binary and text class training
from sklearn import preprocessing
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC


def predictpostinfill(category, model, df_test_fillfeatures, \
                      columnslist = []):
  
  
  #a reasonable extension of this funciton would be to allow ML inference with \
  #other ML architectures such a SVM or something SGD based for instance
  
  #convert dataframes to numpy arrays
#   np_train_filltrain = df_train_filltrain.values
#   np_train_filllabel = df_train_filllabel.values
#   np_train_fillfeatures = df_train_fillfeatures.values
  np_test_fillfeatures = df_test_fillfeatures.values
  
  #ony run the following if we have any rows needing infill
#   if df_train_fillfeatures.shape[0] > 0:
  #since we don't have df_train_fillfeatures to work with we'll look at the 
  #model which will be set to False if there was no infill model trained
  #if model[0] != False:
  if model != False:

    if category == 'nmbr':

#       #train linear regression model using scikit-learn for numerical prediction
#       #model = LinearRegression()
#       #model = PassiveAggressiveRegressor(random_state = randomseed)
#       #model = Ridge(random_state = randomseed)
#       #model = RidgeCV()
#       #note that SVR doesn't have an argument for random_state
#       model = SVR()
#       model.fit(np_train_filltrain, np_train_filllabel)    
      
      
#       #predict infill values
#       np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
#       df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])

#       print('category is nmbr, df_traininfill is')
#       print(df_traininfill)
  
  
  
    if category == 'bxcx':

      
#       model = SVR()
#       model.fit(np_train_filltrain, np_train_filllabel)   
      
#       #predict infill values
#       np_traininfill = model.predict(np_train_fillfeatures)

      
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
#       df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])     

  
    if category == 'bnry':

#       #train logistic regression model using scikit-learn for binary classifier
#       #model = LogisticRegression()
#       #model = LogisticRegression(random_state = randomseed)
#       #model = SGDClassifier(random_state = randomseed)
#       model = SVC(random_state = randomseed)
      
#       model.fit(np_train_filltrain, np_train_filllabel)

#       #predict infill values
#       np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        np_testinfill = np.array([0])

      #convert infill values to dataframe
#       df_traininfill = pd.DataFrame(np_traininfill, columns = ['infill'])
      df_testinfill = pd.DataFrame(np_testinfill, columns = ['infill'])


    if category == 'text':

#       #first convert the one-hot encoded set via argmax to a 1D array
#       np_train_filllabel_argmax = np.argmax(np_train_filllabel, axis=1)

#       #train logistic regression model using scikit-learn for binary classifier
#       #with multi_class argument activated
#       #model = LogisticRegression()
#       #model = SGDClassifier(random_state = randomseed)
#       model = SVC(random_state = randomseed)
      
#       model.fit(np_train_filltrain, np_train_filllabel_argmax)

#       #predict infill values
#       np_traininfill = model.predict(np_train_fillfeatures)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = model.predict(np_test_fillfeatures)
      else:
        #this needs to have same number of columns as text category
        np_testinfill = np.zeros(shape=(1,len(columnslist)))

      #convert the 1D arrary back to one hot encoding
#       labelbinarizertrain = preprocessing.LabelBinarizer()
#       labelbinarizertrain.fit(np_traininfill)
#       np_traininfill = labelbinarizertrain.transform(np_traininfill)
      
      #only run following if we have any test rows needing infill
      if df_test_fillfeatures.shape[0] > 0:
        labelbinarizertest = preprocessing.LabelBinarizer()
        labelbinarizertest.fit(np_testinfill)
        np_testinfill = labelbinarizertest.transform(np_testinfill)



      #run function to ensure correct dimensions of re-encoded classifier array
#       np_traininfill = labelbinarizercorrect(np_traininfill, columnslist)
      
      if df_test_fillfeatures.shape[0] > 0:
        np_testinfill = labelbinarizercorrect(np_testinfill, columnslist)


      #convert infill values to dataframe
#       df_traininfill = pd.DataFrame(np_traininfill, columns = [columnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [columnslist]) 


#       print('category is text, df_traininfill is')
#       print(df_traininfill)

    if category == 'date':

      #create empty sets for now
      #an extension of this method would be to implement a comparable infill \
      #method for the time category, based on the columns output from the \
      #preprocessing
#       df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 
      
#       model = False

#       print('category is text, df_traininfill is')
#       print(df_traininfill)
  
  
  #else if we didn't have any infill rows let's create some plug values
  else:
    
    if category == 'text':
#       np_traininfill = np.zeros(shape=(1,len(columnslist)))
      np_testinfill = np.zeros(shape=(1,len(columnslist)))
#       df_traininfill = pd.DataFrame(np_traininfill, columns = [columnslist])
      df_testinfill = pd.DataFrame(np_testinfill, columns = [columnslist]) 
    
    else :
#       df_traininfill = pd.DataFrame({'infill' : [0]}) 
      df_testinfill = pd.DataFrame({'infill' : [0]}) 
  
#     model = False
  
  return df_testinfill




# 8) update postmunge function for subsequent test data

In [0]:
#postmunge(df_test, testID_column, postprocess_dict) Function that when fed a \
#test data set coresponding to a previously processed train data set which was \
#processed using the automunge function automates the process \
#of evaluating each column for determination and applicaiton of appropriate \
#preprocessing. Takes as arguement pandas dataframes of test data \
#(mdf_test), a string identifying the ID column for test (testID_column), a \
#dictionary containing keys for the processing which had been generated by the \
#original processing of the coresponding train set using automunge function. \
#Returns following sets as numpy arrays: 
#test, testID, labelsencoding_dict, finalcolumns_test

#Requires consistent column naming and order as original train set pre \
#application of automunge. Requires postprocess_dict from original applicaiton. \
#Currently assumes coinbsistent columns carved out from application of munging \
#from original automunge, a potential future extension is to allow for additional \
#columns to be excluded from processing.



def postmunge(postprocess_dict, df_test, testID_column = False):
  
  
  #my understanding is it is good practice to convert any None values into NaN \
  #so I'll just get that out of the way
  df_test.fillna(value=float('nan'), inplace=True)
  
  #extract the ID columns from test set
  if testID_column != False:
    df_testID = pd.DataFrame(df_test[testID_column])
    del df_test[testID_column]
  
  #confirm consistency of train an test sets
  
  #check number of columns is consistent
  if len(postprocess_dict['origtraincolumns'])!= df_test.shape[1]:
    print("error, different number of original columns in train and test sets")
    return
  
  #check column headers are consistent (this works independent of order)
  columns_train_set = set(postprocess_dict['origtraincolumns'])
  columns_test_set = set(list(df_test))
  if columns_train_set != columns_test_set:
    print("error, different column labels in the train and test set")
    return
  
  #check order of column headers are consistent
  columns_train = postprocess_dict['origtraincolumns']
  columns_test = list(df_test)
  if columns_train != columns_test:
    print("error, different order of column labels in the train and test set")
    return
  
  #create an empty dataframe to serve as a store for each column's NArows
  #the column id's for this df will follow convention from NArows of 
  #column+'_NArows' for each column in columns_train
  #these are used in the ML infill methods
  #masterNArows_train = pd.DataFrame()
  masterNArows_test = pd.DataFrame()

    
  #For each column, determine appropriate processing function
  #processing function will be based on evaluation of train set
  for column in columns_train:

    
    #we're only going to process columns that weren't in our excluded set
    #note a foreseeable workflow would be for there to be additional\
    #columns desired for exclusion in post processing, consider adding\
    #additional excluded columns as future extensionl
    if column not in postprocess_dict['excludetransformscolumns']:

 
      category = evalcategory(df_test, column)


      
      #ok postprocess_dict stores column data by the key of column names after\
      #they have gone through our pre-processing functions, which means the \
      #column will match for categories of 'nmbr' and 'bnry', but for the \
      #categories of 'date' and 'text' the act of processing will have \
      #created new columns and deleted the original column - so since we are \
      #currently walking through the original column names we'll need to \
      #pull a post-process column name for the associated columns to serve as \
      #a key for our postprocess_dict which we'll call columnkey. Also the  \
      #original category from train set (traincategory) will be accessed to \
      #serve as a check for consistency between train and test sets.
      traincategory = False
      
      for postprocesscolumn in postprocess_dict['finalcolumns_train']:
        
        
        
        if postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'text':
          if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
          and postprocesscolumn[-5:] != '_NArw':
            traincategory = 'text'
            columnkey = postprocesscolumn
            break
            
        elif postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'date':
          if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
          and postprocesscolumn[-5:] != '_NArw':
            traincategory = 'date'
            columnkey = postprocesscolumn
            break
            
        elif postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'bxcx':
          if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
          and postprocesscolumn[-5:] != '_NArw':
            traincategory = 'bxcx'
            columnkey = postprocesscolumn
            break
            
        elif postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'bnry':
          if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
          and postprocesscolumn[-5:] != '_NArw':
            traincategory = 'bnry'
            columnkey = postprocesscolumn
            break
            
        elif postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'nmbr':
          if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
          and postprocesscolumn[-5:] != '_NArw':
            traincategory = 'nmbr'
            columnkey = postprocesscolumn
            break
        
        elif traincategory == False:
          traincategory = 'null'
          break
#         elif postprocess_dict['column_dict'][postprocesscolumn]['category'] == 'null':
#           if column == postprocess_dict['column_dict'][postprocesscolumn]['origcolumn'] \
#           and postprocesscolumn[-5:] != '_NArw':
#             traincategory = 'null'
#             columnkey = postprocesscolumn 
#             break
            


      
      #for the special case of train category = bxcx and test category = nmbr
      #(meaning there were no negative values in train but there were in test)
      #we'll resolve by clipping all test values that were <0.001 and setting to 
      #NaN then resetting the test category to bxcx to be consistent with train
      if traincategory == 'bxcx' and category == 'nmbr':
        
        #convert all values to either numeric or NaN
        df_test[column] = pd.to_numeric(df_test[column], errors='coerce')
        
        
        df_test[column] = df_test[column].mask(df_test[column] < 0.001)
        category = 'bxcx'
        print('Note that values < 0.001 found in test set were reset to NaN')
        print('to allow consistent box-cox transform as train set.')
      
      #one more special case, if train category is nmbr and test category is bxcx
      #default test category to nmbr
      if traincategory == 'nmbr' and category == 'bxcx':
        category = 'nmbr'
      
      
      #let's make sure the category is consistent between train and test sets
      if category != traincategory:
        print('error - different category between train and test sets for column ',\
              column)
      
      
      #here we'll delete any columns that returned a 'null' category
      if category == 'null':
        df_test = df_test.drop([column], axis=1)
        
      #so if we didn't delete the column let's proceed
      else:
        
        #create NArows (column of True/False where True coresponds to missing data)
        testNArows = NArows(df_test, column, category)
        
        #now append that NArows onto a master NA rows df
        masterNArows_test = pd.concat([masterNArows_test, testNArows], axis=1)

        #(now normalize as would normally)

        #for binary class use the train majority field for missing plug value
        if category == 'bnry':
          binary_missing_plug = postprocess_dict['column_dict'][columnkey]['normalization_dict']['missing']
          
        #apply appropriate processing function to this column based on the result
        #original bnry processing function still works since only had one df input
        if category == 'bnry':
          df_test, _1, _2 = process_binary_class(df_test, column, binary_missing_plug)
          
        #for nmbr category process test set with function postprocess_numerical_class
        if category == 'nmbr':
          df_test = postprocess_numerical_class(df_test, column, \
                                                postprocess_dict['column_dict'][columnkey]['normalization_dict']['mean'], \
                                                postprocess_dict['column_dict'][columnkey]['normalization_dict']['std'])
        
        #for bxcx category processing
        if category == 'bxcx':          
          
          
          df_test, nmbrcolumns, nmbrnormalization_dict, _ = \
          process_bxcx_class(df_test, column, bxcx_lmbda = \
                             postprocess_dict['column_dict'][columnkey]['normalization_dict']['bxcx_lmbda'], \
                             trnsfrm_mean = postprocess_dict['column_dict'][columnkey]['normalization_dict']['trnsfrm_mean'], \
                             trnsfrm_std = postprocess_dict['column_dict'][columnkey]['normalization_dict']['trnsfrm_std'])

        
        #for text category process test set with function postprocess_text_class
        if category == 'text':

          
          df_test = postprocess_text_class(df_test, column, postprocess_dict['column_dict'][columnkey]['columnslist'])
        
        #for date category process test set with function postprocess_date_class
        if category == 'date':
          df_test = postprocess_time_class(df_test, column, \
                                           postprocess_dict['column_dict'][columnkey]['columnslist'], \
                                           postprocess_dict['column_dict'][columnkey]['normalization_dict'])
          
        
  #now that we've pre-processed all of the columns, let's run through them again\
  #using ML to derive plug values for the previously missing cells
  
  #if MLinfill == True
  if postprocess_dict['MLinfill'] == True:
    
    
    #now let's create a list of columns just like we did in automunge
    columns_test_ML = list(df_test)
    
    iteration = 0
    #while iteration < infilliterate:
    while iteration < postprocess_dict['infilliterate']:
      
          
      #since we're reusing the text_dict and date_dict from our original automunge
      #we're going to need to re-initialize the infillcomplete markers
      #actually come to this of it we need to go back to automunge and do this
      #for the MLinfill iterations as well
      
      #re-initialize the infillcomplete marker in column _dict's
      for key in postprocess_dict['column_dict']:
        postprocess_dict['column_dict'][key]['infillcomplete'] = False

      
      
      for column in columns_test_ML:

        
        #we're only going to process columns that weren't in our excluded set
        #if column not in excludetransformscolumns:
        if column not in postprocess_dict['excludetransformscolumns'] \
        and column[-5:] != '_NArw':
          
          #If column id is found in the text_dict then will require different \
          #type of address since this category won't be found in our \
          #mastercategory_dict and we'll need to apply the ML infill to the \
          #collective group of columns from the associated textcolumns array.
          
          #if column in text_dict:
          #if column in postprocess_dict['text_dict']:
          if column[-5:] == '_text':
            
            #check the status of dictionary's infillcomplete marker for this column
            #if postprocess_dict['text_dict'][column]['infillcomplete'] == False:
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
              
              #pull this column's textcolumns array
              textcolumns = \
              postprocess_dict['column_dict'][column]['columnslist']
              
              category = postprocess_dict['column_dict'][column]['category']
              origcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              columnslist = postprocess_dict['column_dict'][column]['columnslist']
              categorylist = postprocess_dict['column_dict'][column]['categorylist']
              
              #now let's apply our functions for ML infill
              
              
              df_test_fillfeatures = \
              createpostMLinfillsets(df_test, column, pd.DataFrame(masterNArows_test[origcolumn+'_NArows']), \
                                     category, columnslist = columnslist, \
                                     categorylist = categorylist)
              
              #predict infill values using defined function predictpostinfill(.)
              df_testinfill = \
              predictpostinfill(category, postprocess_dict['column_dict'][column]['infillmodel'], \
                                df_test_fillfeatures, columnslist = columnslist)
              
              #it's a quirk of the ML models that if we don't train the
              #train set model on any features, that we won't be able to apply
              #the model to predict the test set infill. 
              #For now we'll only use insertilnfill if we had
              #some missing points in the train set, a future extension would be
              #to update our createMLinfillsets and predictinfill to create 
              #some arbitrary features to train the infill on train set for
              #cases where there are NaN values in test set but not in train
              #such that we could insert infill for missing values in test set.              
              
              #if model != False:
              if postprocess_dict[column]['infillmodel'] != False:
                
                df_test = insertinfill(df_test, column, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[origcolumn+'_NArows']), \
                                       columnslist = columnslist)
              
              #now change the infillcomplete marker in the text_dict for each \
              #associated text column
              for textcolumnname in categorylist:
                postprocess_dict['column_dict'][textcolumnname]['infillcomplete'] == True
                                                    
                
          #If column id found in the bxcx_dict then we'll address with the
          #box-cox methods
          #elif column in postprocess_dict['bxcx_dict']:
          if column[-5:] == '_bxcx':
            
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
              
              #pull this column's columnslist
              #for the record I know this is a list not an array
              nmbrcolumns = postprocess_dict['column_dict'][column]['columnslist']
              categorylist = postprocess_dict['column_dict'][column]['categorylist']
              
              #grab the original column name prior to preprocessing
              nmbrorigcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              
              #now get the transformed column name for the two categories of nmbr and bxcx
              column_bxcx = nmbrorigcolumn + '_bxcx'
              category_bxcx = 'bxcx'
              column_nmbr = nmbrorigcolumn + '_nmbr'
              category_nmbr = 'nmbr'
              
              #now let's apply our functions for ML infill
              
              #createMLinfillsets
              
              df_test_fillfeatures = \
              createpostMLinfillsets(df_test, column, \
                                     pd.DataFrame(masterNArows_test[nmbrorigcolumn + '_NArows']), \
                                     category_bxcx, columnslist = nmbrcolumns, \
                                     categorylist = categorylist)
                                     
              #predict infill values using defined function predictinfill(.)
              #first we'll address the bxcx transformed column



              df_testinfill_bxcx = predictpostinfill(category_bxcx, \
                                                     postprocess_dict['column_dict'][column_bxcx]['infillmodel'], \
                                                     df_test_fillfeatures, columnslist = [])
              
              #now we'll address the nmbr column
              df_testinfill_nmbr = predictpostinfill(category_nmbr, \
                                                     postprocess_dict['column_dict'][column_nmbr]['infillmodel'], \
                                                     df_test_fillfeatures, columnslist = [])
              
              #apply the function insertinfill(.) to insert missing value predicitons \
              #to df's associated column

              #only use insertilnfill if we had some missing points in the train set
              if postprocess_dict['column_dict'][column]['infillmodel'] != False:
                
                              
                df_test = insertinfill(df_test, column_bxcx, df_testinfill_bxcx, \
                                       category_bxcx, \
                                       pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                       columnslist = [])
                
                
                df_test = insertinfill(df_test, column_nmbr, df_testinfill_nmbr, \
                                       category_nmbr, \
                                       pd.DataFrame(masterNArows_test[nmbrorigcolumn+'_NArows']), \
                                       columnslist = [])

              #now change the infillcomplete marker in the text_dict for each \
              #associated text column
              for nmbrcolumnname in nmbrcolumns:
                postprocess_dict['column_dict'][nmbrcolumnname]['infillcomplete'] == True
                
                
          #If column id is found in the date_dict then will require different \
          #type of address since this category won't be found in our \
          #mastercategory_dict and we'll need to apply the ML infill to the \
          #collective group of columns from the associated datecolumns array. \
          #The development of this address for date columns is a future extension.
          #elif column in postprocess_dict['date_dict']:
          elif column[-5:] == '_date':

            #this too shall be a future extension.
            pass
          
          #elif column in bnry_dict:
          elif column[-5:] == '_bnry':
          
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
            
              #grab a few column values from postprocess_dict  
              category = postprocess_dict['column_dict'][column]['category']
              origcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              columnslist = postprocess_dict['column_dict'][column]['columnslist']
              categorylist = postprocess_dict['column_dict'][column]['categorylist']

              #create MLinfill set using defined function createpostMLinfillsets(.)
              df_test_fillfeatures = \
              createpostMLinfillsets(df_test, column, pd.DataFrame(masterNArows_test[origcolumn + '_NArows']), \
                                     category, columnslist = columnslist, \
                                     categorylist = categorylist)
              
              #predict infill values using defined function predictpostinfill(.)
              df_testinfill = \
              predictpostinfill(category, postprocess_dict['column_dict'][column]['infillmodel'], \
                                df_test_fillfeatures, columnslist = columnslist)
              
              #(only insert infill to test set if we had NA rows in train set)
              #note comments above in text class for potential future expansion
              if postprocess_dict['column_dict'][column]['infillmodel'] != False:
                
                df_test = insertinfill(df_test, column, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[origcolumn + '_NArows']), \
                                       columnslist = columnslist, \
                                       categorylist = categorylist)
              
          #elif column in nmbr_dict:
          elif column[-5:] == '_nmbr':    
          
            #check the status of dictionary's infillcomplete marker for this column
            if postprocess_dict['column_dict'][column]['infillcomplete'] == False:
            
              #grab a few column values from postprocess_dict  
              category = postprocess_dict['column_dict'][column]['category']
              origcolumn = postprocess_dict['column_dict'][column]['origcolumn']
              columnslist = postprocess_dict['column_dict'][column]['columnslist']
              categorylist = postprocess_dict['column_dict'][column]['categorylist']
              
              #create MLinfill set using defined function createpostMLinfillsets(.)
              df_test_fillfeatures = \
              createpostMLinfillsets(df_test, column, pd.DataFrame(masterNArows_test[origcolumn + '_NArows']), \
                                     category, columnslist = columnslist, \
                                     categorylist = categorylist)
              
              #predict infill values using defined function predictpostinfill(.)
              df_testinfill = \
              predictpostinfill(category, postprocess_dict['column_dict'][column]['infillmodel'], \
                                df_test_fillfeatures, columnslist = [])
              
              #(only insert infill to test set if we had NA rows in train set)
              #note comments above in text class for potential future expansion
              if postprocess_dict['column_dict'][column]['infillmodel'] != False:
                
                df_test = insertinfill(df_test, column, df_testinfill, category, \
                                       pd.DataFrame(masterNArows_test[origcolumn + '_NArows']), \
                                       columnslist = [])
          
          

          
      iteration += 1          

  #here's a list of final column names saving here since the translation to \
  #numpy arrays scrubs the column names
  finalcolumns_test = list(df_test)
  
  #global processing to test set including conversion to numpy array
  np_test = df_test.values
  
  if testID_column != False:
    np_testID = df_testID.values
    testID = np_testID
  else:
    testID = []
    
  test = np_test
  
  labelsencoding_dict = postprocess_dict['labelsencoding_dict']
  
  
  return test, testID, labelsencoding_dict, finalcolumns_test
  


# 9) Test our functions

In [0]:

#create sample test and train data for demonstration purposes

#train data set from list of dictionaries
#24 rows
# train = [{'ID' : 101, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
#          {'ID' : 102, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
#          {'ID' : 103, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
#          {'ID' : 104, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
#          {'ID' : 105, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
#          {'ID' : 106, 'number': 'Q', 'Y-N': 'N', 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
#          {'ID' : 107, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
#          {'ID' : 108, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
#          {'ID' : 109, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
#          {'ID' : 110, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
#          {'ID' : 111, 'number': -1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
#          {'ID' : 112, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
#          {'ID' : 113, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
#          {'ID' : 114, 'number': 2, 'Y-N': None, 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
#          {'ID' : 115, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
#          {'ID' : 116, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
#          {'ID' : 117, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
#          {'ID' : 118, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'}]

#revised with all positive number column for power transform
train = [{'ID' : 101, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 102, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 103, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 104, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 105, 'number': 1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 106, 'number': 'Q', 'Y-N': 'N', 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 107, 'number': 1, 'Y-N': None, 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 108, 'number': 2, 'Y-N': 'N', 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 109, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 110, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 111, 'number': 1, 'Y-N': None, 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 112, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'},
         {'ID' : 113, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : '2/12/18', 'label': 'cat'}, 
         {'ID' : 114, 'number': 2, 'Y-N': None, 'shape': 'square', 'date' : 'August 12, 2016', 'label': 'dog'}, 
         {'ID' : 115, 'number': None, 'Y-N': 'Y', 'shape': 'circle', 'date' : None, 'label': 'cat'},
         {'ID' : 116, 'number': 3.1, 'Y-N': None, 'shape': 'square', 'date' : 'July 4, 2016', 'label': 'cat'}, 
         {'ID' : 117, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'Jul 4, 2018', 'label': 'dog'}, 
         {'ID' : 118, 'number': 'Q', 'Y-N': None, 'shape': 'oval', 'date' : '2015', 'label': 'dog'}]

#convert train data to pandas dataframe
df_train = pd.DataFrame(train)


#test data set from list of dictionaries
#21 rows
# test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID': 2, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 6, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
#         {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID' : 9, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 13, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
#         {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID' : 16, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 20, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID': 2, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 6, 'number': 10, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 9, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 13, 'number': 10, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 16, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 20, 'number': 10, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

#convert test data to pandas dataframe
df_test = pd.DataFrame(test)



In [14]:
#apply automunge
#this application is primarily to serve as a quick check for bugs prior to more\
#computationally expensive applicaitons in cases of some update to automunge
start = timer()

train, trainID, labels, validation, validationID, validationlabels, test, \
testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
postprocess_dict = \
automunge(df_train, df_test, labels_column = 'label', trainID_column = 'ID', \
         testID_column = 'ID', powertransform = True, MLinfill = True, infilliterate=1, \
         randomseed = 42, excludetransformscolumns = [])

end = timer()

print('seconds elapsed = ', end - start)

seconds elapsed =  0.24109430099997553


In [15]:
#validate the output
print('train shape =            ', train.shape)
print('trainID shape =          ', trainID.shape)
print('labels shape =           ', labels.shape)
print('validation shape =       ', validation.shape)
print('validationID shape =     ', validationID.shape)
print('validationlabels shape = ', validationlabels.shape)
print('test shape =             ', test.shape)
print('testID shape =           ', testID.shape)
print('labelsencoding_dict = ')
print(labelsencoding_dict)
print('finalcolumns_train = ')
print(finalcolumns_train)
print('finalcolumns_test = ')
print(finalcolumns_test)

train shape =             (14, 16)
trainID shape =           (14, 1)
labels shape =            (14, 2)
validation shape =        (4, 16)
validationID shape =      (4, 1)
validationlabels shape =  (4, 2)
test shape =              (21, 16)
testID shape =            (21, 1)
labelsencoding_dict = 
{'bnry': {0: 'cat'}}
finalcolumns_train = 
['shape_NArw', 'shape_circle', 'shape_oval', 'shape_square', 'Y-N_NArw', 'Y-N_bnry', 'date_NArw', 'date_year', 'date_month', 'date_day', 'date_hour', 'date_minute', 'date_second', 'number_NArw', 'number_bxcx', 'number_nmbr']
finalcolumns_test = 
['shape_NArw', 'shape_circle', 'shape_oval', 'shape_square', 'Y-N_NArw', 'Y-N_bnry', 'date_NArw', 'date_year', 'date_month', 'date_day', 'date_hour', 'date_minute', 'date_second', 'number_NArw', 'number_bxcx', 'number_nmbr']


In [0]:
#demonstrate download of postprocess_dict

#save postprocess_dict as pickle object
with open('postprocess_dict.pickle', 'wb') as handle:
    pickle.dump(postprocess_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
#download to local drive
#(code specific to Colaboratory)
files.download('postprocess_dict.pickle')


In [0]:
#demonstrate upload of postprocess_dict

#upload postprocess_dict from local drive
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving postprocess_dict.pickle to postprocess_dict (1).pickle
User uploaded file "postprocess_dict.pickle" with length 6564 bytes


In [0]:
#acces uploaded file with pickle
#(code specific to Colaboratory)
with open('postprocess_dict.pickle', 'rb') as handle:
    postprocess_dict_upload = pickle.load(handle)

In [0]:
#test data set from list of dictionaries
#21 rows
# test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID': 2, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 6, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
#         {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID' : 9, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 13, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
#         {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
#         {'ID' : 16, 'number': -1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
#         {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
#         {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
#         {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
#         {'ID' : 20, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
#         {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

#revised test data with all positive values in number column for power transform
test = [{'ID' : 1, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID': 2, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 3,'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 4, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 5, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 6, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 7, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 8, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 9, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 10, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 11, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 12, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 13, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 14, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'},
        {'ID' : 15, 'number': 2.1, 'Y-N': 'N', 'shape': 'square', 'date' : '4/14/18'}, 
        {'ID' : 16, 'number': 1, 'Y-N': 'N', 'shape': None, 'date' : 'August 12, 2016'},
        {'ID' : 17, 'number': 1, 'Y-N': 'Y', 'shape': 'circle', 'date' : 'July 4, 2018'}, 
        {'ID' : 18, 'number': None, 'Y-N': 'Y', 'shape': 'square', 'date' : None}, 
        {'ID' : 19, 'number': 3, 'Y-N': None, 'shape': 'circle', 'date' : 'Aug 31, 2018'}, 
        {'ID' : 20, 'number': 0, 'Y-N': 'N', 'shape': 'octogon', 'date' : '2017'}, 
        {'ID' : 21, 'number': 'Q', 'Y-N': 'Y', 'shape': 'square', 'date' : 'Jan 1, 2019'}]

#convert test data to pandas dataframe
df_test = pd.DataFrame(test)


In [18]:
#apply postmunge
#this application is primarily to serve as a quick check for bugs prior to more\
#computationally expensive applicaitons in cases of some update to automunge

start = timer()

# test, testID, labelsencoding_dict, finalcolumns_test = \
# postmunge(postprocess_dict_upload, df_test, testID_column = 'ID')

test, testID, labelsencoding_dict, finalcolumns_test = \
postmunge(postprocess_dict, df_test, testID_column = 'ID')

end = timer()

print('seconds elapsed = ', end - start)

Note that values < 0.001 found in test set were reset to NaN
to allow consistent box-cox transform as train set.
seconds elapsed =  0.11346654699991632


In [19]:
#validate the output
print('test shape =             ', test.shape)
print('testID shape =           ', testID.shape)
print('labelsencoding_dict = ')
print(labelsencoding_dict)
print('finalcolumns_test = ')
print(finalcolumns_test)

test shape =              (21, 16)
testID shape =            (21, 1)
labelsencoding_dict = 
{'bnry': {0: 'cat'}}
finalcolumns_test = 
['shape_NArw', 'shape_circle', 'shape_oval', 'shape_square', 'Y-N_NArw', 'Y-N_bnry', 'date_NArw', 'date_year', 'date_month', 'date_day', 'date_hour', 'date_minute', 'date_second', 'number_NArw', 'number_bxcx', 'number_nmbr']


In [21]:
#Now let's try a larger dataset, the Titanic dataset from Kaggle
#available here: https://www.kaggle.com/c/titanic/data
#(which I will upload form my local hard drive)
#for more on data imports in Colaboratory see my medium post 
#https://medium.com/@_NicT_/colaboratorys-free-gpu-72ebc9272933
#Following is as presented in the Colaboratory tutorial notebook
#Once run this will allow you to manually select the path on local drive for file you wish to upload

#import titanic train data
from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving train.csv to train.csv
User uploaded file "train.csv" with length 60302 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
titanic_train_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
#titanic_train_dforig.head()

In [23]:
#import titanic test data
from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving test.csv to test.csv
User uploaded file "test.csv" with length 28210 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
titanic_test_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
#titanic_test_dforig.head()

In [0]:
#initialize data
titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()


#Now there are certain aspects of feature engineering that our automunge won't address
#for example one could extract from the Mrs/Ms/Miss designation in the Name \
#column if a female is married. From Cabin field perhaps we could infer what \
#deck passenger was on or whether they even had a cabin. This type of evaluation \
#would need to be done prior to applicaiton of automunge. Because each column is \
#unique there won't be any learning for Cabin, Name, or Ticket I expect so we'll \
#go ahead and delete those rows for our demonstration. It is certainly \
#feasible that there is some feature buried in these columns that can be \
#extracted prior to applicaiton of automunge. PassengerId will serve as ID column.

titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [26]:
#now let's run our automunge function and see how we did, first we'll try \
#without the MLinfill:

start = timer()

train, trainID, labels, validation, validationID, validationlabels, test, \
testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
postprocess_dict = \
automunge(titanic_train_df, titanic_test_df, labels_column = 'Survived', \
          trainID_column = 'PassengerId', testID_column = 'PassengerId', \
          MLinfill = False)

end = timer()

print('seconds elapsed = ', end - start)


seconds elapsed =  1.0846949470001164


In [27]:
print('train shape =            ', train.shape)
print('trainID shape =          ', trainID.shape)
print('labels shape =           ', labels.shape)
print('validation shape =       ', validation.shape)
print('validationID shape =     ', validationID.shape)
print('validationlabels shape = ', validationlabels.shape)
print('test shape =             ', test.shape)
print('testID shape =           ', testID.shape)
print('labelsencoding_dict = ')
print(labelsencoding_dict)
print('finalcolumns_train = ')
print(finalcolumns_train)
print('finalcolumns_test = ')
print(finalcolumns_test)

train shape =             (712, 18)
trainID shape =           (712, 1)
labels shape =            (712, 2)
validation shape =        (179, 18)
validationID shape =      (179, 1)
validationlabels shape =  (179, 2)
test shape =              (418, 18)
testID shape =            (418, 1)
labelsencoding_dict = 
{'bnry': {0: 0}}
finalcolumns_train = 
['Embarked_C', 'Embarked_NArw', 'Embarked_Q', 'Embarked_S', 'Pclass_NArw', 'Pclass_bxcx', 'Pclass_nmbr', 'Sex_NArw', 'Sex_bnry', 'Age_NArw', 'Age_bxcx', 'Age_nmbr', 'SibSp_NArw', 'SibSp_nmbr', 'Parch_NArw', 'Parch_nmbr', 'Fare_NArw', 'Fare_nmbr']
finalcolumns_test = 
['Embarked_C', 'Embarked_NArw', 'Embarked_Q', 'Embarked_S', 'Pclass_NArw', 'Pclass_bxcx', 'Pclass_nmbr', 'Sex_NArw', 'Sex_bnry', 'Age_NArw', 'Age_bxcx', 'Age_nmbr', 'SibSp_NArw', 'SibSp_nmbr', 'Parch_NArw', 'Parch_nmbr', 'Fare_NArw', 'Fare_nmbr']


In [0]:
#re-initialize test data
#titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()


#titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [29]:
#apply postmunge

start = timer()

test, testID, labelsencoding_dict, finalcolumns_test = \
postmunge(postprocess_dict, titanic_test_df, testID_column = 'PassengerId')

# test, testID, labelsencoding_dict, finalcolumns_test = \
# postmunge(postprocess_dict_upload, titanic_test_df, testID_column = 'PassengerId')

end = timer()

print('seconds elapsed = ', end - start)

seconds elapsed =  0.32638505100021575


In [0]:
#re-initialize data
titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()

titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [31]:
#now let's run our automunge function and see how we did, now we'll try \
#with the MLinfill included:

start = timer()

train, trainID, labels, validation, validationID, validationlabels, test, \
testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
postprocess_dict = \
automunge(titanic_train_df, titanic_test_df, labels_column = 'Survived', \
          trainID_column = 'PassengerId', testID_column = 'PassengerId', \
          MLinfill = True)

end = timer()

print('seconds elapsed = ', end - start)

seconds elapsed =  1.2998147769999377


In [0]:
#re-initialize test data
#titanic_train_df = titanic_train_dforig.copy()
titanic_test_df = titanic_test_dforig.copy()


#titanic_train_df = titanic_train_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)
titanic_test_df = titanic_test_df.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [33]:
#apply postmunge

start = timer()

test, testID, labelsencoding_dict, finalcolumns_test = \
postmunge(postprocess_dict, titanic_test_df, testID_column = 'PassengerId')

# test, testID, labelsencoding_dict, finalcolumns_test = \
# postmunge(postprocess_dict_upload, titanic_test_df, testID_column = 'PassengerId')

end = timer()

print('seconds elapsed = ', end - start)

seconds elapsed =  0.4100760760002231


In [34]:
#ok let's try with a different set, how about the Kaggel house prices competition
#available here: https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data
#(which I will upload form my local hard drive)
#for more on data imports in Colaboratory see my medium post 
#https://medium.com/@_NicT_/colaboratorys-free-gpu-72ebc9272933
#Following is as presented in the Colaboratory tutorial notebook
#Once run this will allow you to manually select the path on local drive for file you wish to upload

#upload train data
from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving train.csv to train (1).csv
User uploaded file "train.csv" with length 460676 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
house_train_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
#house_train_dforig.head()

In [36]:
#upload test data
from google.colab import files
uploaded = files.upload()
for train in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(name=train, length=len(uploaded[train])))


Saving test.csv to test (1).csv
User uploaded file "test.csv" with length 451405 bytes


In [0]:
#Here is some additional detail for converting 
#the resulting upload into a dataframe

from io import BytesIO
house_test_dforig = pd.read_csv(BytesIO(uploaded[train]), encoding='latin-1')
#house_test_dforig.head()

In [0]:
#Ok some bug with the fireplace column so let's just delete that one from our sets
#(I believe the evalcategory(.) function is determining different category \
#between train and test sets, fixing this bug is future extension.)
house_train_df = house_train_dforig.copy()
house_train_df = house_train_df.drop(['FireplaceQu'], axis=1)

house_test_df = house_test_dforig.copy()
house_test_df = house_test_df.drop(['FireplaceQu'], axis=1)


In [47]:
#now let's run our automunge function and see how we did, first we'll try \
#without the MLinfill:

start = timer()

train, trainID, labels, validation, validationID, validationlabels, test, \
testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
postprocess_dict = \
automunge(house_train_df, house_test_df, labels_column = 'SalePrice', \
          trainID_column = 'Id', testID_column = 'Id', MLinfill = False)

end = timer()

print('seconds elapsed = ', end - start)

/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
/usr/local/lib/python3.6/dist-packages/scipy/optimize/optimize.py:1850: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


seconds elapsed =  26.274990118000005


In [48]:
print('train shape =            ', train.shape)
print('trainID shape =          ', trainID.shape)
print('labels shape =           ', labels.shape)
print('validation shape =       ', validation.shape)
print('validationID shape =     ', validationID.shape)
print('validationlabels shape = ', validationlabels.shape)
print('test shape =             ', test.shape)
print('testID shape =           ', testID.shape)
print('labelsencoding_dict = ')
print(labelsencoding_dict)
print('finalcolumns_train = ')
print(finalcolumns_train)
print('finalcolumns_test = ')
print(finalcolumns_test)

train shape =             (1168, 354)
trainID shape =           (1168, 1)
labels shape =            (1168, 1)
validation shape =        (292, 354)
validationID shape =      (292, 1)
validationlabels shape =  (292, 1)
test shape =              (1459, 354)
testID shape =            (1459, 1)
labelsencoding_dict = 
{'bxcx': {}}
finalcolumns_train = 
['SaleCondition_NArw', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial', 'SaleType_NArw', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'PavedDrive_NArw', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'GarageCond_Ex', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_NArw', 'GarageCond_Po', 'GarageCond_TA', 'GarageQual_Ex', 'GarageQual_Fa', 'GarageQual_Gd', 'GarageQual_NArw', 'GarageQual_Po', 'GarageQual_TA', 'GarageFinish_Fin', 'GarageFinish_NArw',

In [0]:
#re-initialize the test set
# house_train_df = house_train_dforig.copy()
# house_train_df = house_train_df.drop(['FireplaceQu'], axis=1)

house_test_df = house_test_dforig.copy()
house_test_df = house_test_df.drop(['FireplaceQu'], axis=1)


In [50]:
#apply postmunge

start = timer()

test, testID, labelsencoding_dict, finalcolumns_test = \
postmunge(postprocess_dict, house_test_df, testID_column = 'Id')

# test, testID, labelsencoding_dict, finalcolumns_test = \
# postmunge(postprocess_dict_upload, house_test_df, testID_column = 'Id')

end = timer()

print('seconds elapsed = ', end - start)

error - different category between train and test sets for column  Alley
error - different category between train and test sets for column  PoolQC
error - different category between train and test sets for column  Fence
error - different category between train and test sets for column  MiscFeature
seconds elapsed =  13.238316189000216


In [0]:
#go ahead and ignore these error messages, these are the columns that returned '
#a category of 'null' from applicaiton of evalcagteghory and will have been dropped\
#I'll have to put some thought in to how to better compare the train and test \
#column categories. (to confirm see look at the validation array shapes below)

In [52]:
#validate the output
print('test shape =             ', test.shape)
print('testID shape =           ', testID.shape)
print('labelsencoding_dict = ')
print(labelsencoding_dict)
print('finalcolumns_test = ')
print(finalcolumns_test)

test shape =              (1459, 354)
testID shape =            (1459, 1)
labelsencoding_dict = 
{'bxcx': {}}
finalcolumns_test = 
['SaleCondition_NArw', 'SaleCondition_Abnorml', 'SaleCondition_AdjLand', 'SaleCondition_Alloca', 'SaleCondition_Family', 'SaleCondition_Normal', 'SaleCondition_Partial', 'SaleType_NArw', 'SaleType_COD', 'SaleType_CWD', 'SaleType_Con', 'SaleType_ConLD', 'SaleType_ConLI', 'SaleType_ConLw', 'SaleType_New', 'SaleType_Oth', 'SaleType_WD', 'PavedDrive_NArw', 'PavedDrive_N', 'PavedDrive_P', 'PavedDrive_Y', 'GarageCond_Ex', 'GarageCond_Fa', 'GarageCond_Gd', 'GarageCond_NArw', 'GarageCond_Po', 'GarageCond_TA', 'GarageQual_Ex', 'GarageQual_Fa', 'GarageQual_Gd', 'GarageQual_NArw', 'GarageQual_Po', 'GarageQual_TA', 'GarageFinish_Fin', 'GarageFinish_NArw', 'GarageFinish_RFn', 'GarageFinish_Unf', 'GarageType_2Types', 'GarageType_Attchd', 'GarageType_Basment', 'GarageType_BuiltIn', 'GarageType_CarPort', 'GarageType_Detchd', 'GarageType_NArw', 'Functional_NArw', 'Functiona

In [0]:
#now let's try again with the MLinfill and compare results. I would expect to \
#see hopefully a more pronounced result than the delta from titanic set due \
#to higher frequency of missing data in the set.

In [0]:
#reinitialize the data
house_train_df = house_train_dforig.copy()
house_train_df = house_train_df.drop(['FireplaceQu'], axis=1)

house_test_df = house_test_dforig.copy()
house_test_df = house_test_df.drop(['FireplaceQu'], axis=1)


In [0]:
#full disclosure the below error is unexpected behavior, I'll have to put some
#troubleshooting time in. Might have something to do with the overflow error
#message which is also new. These might be related to the Random Forest model
#to be continued

In [55]:
#now let's run our automunge function and see how we did with the MLinfill:

start = timer()

train, trainID, labels, validation, validationID, validationlabels, test, \
testID, labelsencoding_dict, finalcolumns_train, finalcolumns_test, \
postprocess_dict = \
automunge(house_train_df, house_test_df, labels_column = 'SalePrice', \
          trainID_column = 'Id', testID_column = 'Id', MLinfill = True)

end = timer()

print('seconds elapsed = ', end - start)

/usr/local/lib/python3.6/dist-packages/scipy/stats/morestats.py:901: RuntimeWarning: overflow encountered in square
  llf -= N / 2.0 * np.log(np.sum((y - y_mean)**2. / N, axis=0))
/usr/local/lib/python3.6/dist-packages/scipy/optimize/optimize.py:1850: RuntimeWarning: invalid value encountered in double_scalars
  tmp2 = (x - v) * (fx - fw)


ValueError: ignored

In [0]:
#reinitialize the test data
# house_train_df = house_train_dforig.copy()
# house_train_df = house_train_df.drop(['FireplaceQu'], axis=1)

house_test_df = house_test_dforig.copy()
house_test_df = house_test_df.drop(['FireplaceQu'], axis=1)

In [0]:
#apply postmunge

start = timer()

test, testID, labelsencoding_dict, finalcolumns_test = \
postmunge(postprocess_dict, house_test_df, testID_column = 'Id')

# test, testID, labelsencoding_dict, finalcolumns_test = \
# postmunge(postprocess_dict_upload, house_test_df, testID_column = 'Id')

end = timer()

print('seconds elapsed = ', end - start)

troubleshooting process_bxcx_class function
column =  MSSubClass
bxcx_lmbda =  -0.2108406694646882
after stats .boxcox transform, bxcx_lmbda =  -0.2108406694646882
troubleshooting process_bxcx_class function
column =  LotFrontage
bxcx_lmbda =  0.43778995839151413
after stats .boxcox transform, bxcx_lmbda =  0.43778995839151413
troubleshooting process_bxcx_class function
column =  LotArea
bxcx_lmbda =  0.030946318971935564
after stats .boxcox transform, bxcx_lmbda =  0.030946318971935564
error - different category between train and test sets for column  Alley
troubleshooting process_bxcx_class function
column =  OverallQual
bxcx_lmbda =  0.7622455699934552
after stats .boxcox transform, bxcx_lmbda =  0.7622455699934552
troubleshooting process_bxcx_class function
column =  OverallCond
bxcx_lmbda =  0.41391221535396094
after stats .boxcox transform, bxcx_lmbda =  0.41391221535396094
troubleshooting process_bxcx_class function
column =  YearBuilt
bxcx_lmbda =  22.116548766131825
after stat